# Boxplot Modelos

> Notebook organizado para reprodutibilidade. Edite apenas a célula **CONFIGURAÇÕES**.

In [ ]:
from pathlib import Path
import os

# CONFIGURAÇÕES (edite se necessário)
# A pasta raiz do projeto (por padrão, a pasta acima de /notebooks)
ROOT = Path(os.getenv('CLIMBRA_PROJECT_ROOT', Path.cwd().parent)).resolve()
DATA_DIR = ROOT / 'data'
RAW_DIR  = DATA_DIR / '00_raw'
INT_DIR  = DATA_DIR / '01_intermediate'
FINAL_DIR= DATA_DIR / '02_final'
OUT_DIR  = ROOT / 'outputs'
FIG_DIR  = OUT_DIR / 'figures'
TAB_DIR  = OUT_DIR / 'tables'

for d in [RAW_DIR, INT_DIR, FINAL_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)


In [ ]:
"""
================================================================================
Cálculo de ΔQ/Q (%) por modelo climático e por minibacia (robusto a extremos)
================================================================================

O que este script faz
---------------------
1) Lê o arquivo histórico (1980–2023) e calcula, por minibacia:
   - Série anual: MÉDIA anual (diário -> anual por mean)
   - Nível do período base (Q_pres): MEDIANA das médias anuais (robusta a anos extremos)

2) Para cada arquivo de modelo (2015–2100), e para cada horizonte:
   - Série anual: MÉDIA anual (diário -> anual por mean)
   - Nível do horizonte (Q_fut): MEDIANA das médias anuais no horizonte
   - ΔQ/Q (%) = (Q_fut - Q_pres) / Q_pres * 100

3) Salva um CSV "longo" com:
   ['modelo', 'horizonte', 'minibacia', 'delta_q_q']

4) Gera boxplots por horizonte:
   - Cada boxplot mostra a distribuição espacial (entre minibacias) de ΔQ/Q
     para cada modelo climático.

POR QUE "MÉDIA ANUAL" + "MEDIANA ENTRE ANOS"?
---------------------------------------------
- MÉDIA ANUAL preserva o conceito hidrológico de vazão média anual.
- MEDIANA entre anos reduz a influência de anos muito extremos e fornece
  um valor típico do período (robusto), ideal para comparações entre períodos.

Observação importante
---------------------
Este script NÃO executa testes de tendência (MK/Sen). Ele apenas calcula ΔQ/Q
para boxplots e para uso posterior em mapas/estatísticas.
================================================================================
"""

import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

# ---------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------
# pasta raiz dos resultados
root_dir = Path(r"E:\RESULTADOS\SSP5_85")

# pasta onde estão os arquivos de projeções futuras (modelos)
base_dir = root_dir / "Qtudo_Fut_Cenario 1"

# arquivo histórico/base (1980–2023)
# ATENÇÃO: mantenha o nome correto do seu arquivo base
obs_file = root_dir / "Qtudo_Pres" / "Observado.txt"

# pasta de saída (CSV + figuras)
output_root = root_dir / "BoxPlot_Modelos"
output_root.mkdir(parents=True, exist_ok=True)

# padrão para os arquivos de modelos
model_pattern = "*-ssp585.txt"  # ajuste se necessário (ex.: "*-ssp245.txt")

# período de referência (histórico)
ref_start = "1980-01-01"
ref_end   = "2023-12-31"

# horizontes futuros
horizons = {
    "2015-2040": ("2015-01-01", "2040-12-31"),
    "2041-2070": ("2041-01-01", "2070-12-31"),
    "2071-2100": ("2071-01-01", "2100-12-31"),
    "2015-2100": ("2015-01-01", "2100-12-31"),
}

# datas completas esperadas nos arquivos
OBS_START, OBS_END = "1980-01-01", "2023-12-31"
FUT_START, FUT_END = "2015-01-01", "2100-12-31"

# proteção contra divisão por zero
EPS = 1e-12

# ---------------------------------------------------
# FUNÇÕES AUXILIARES
# ---------------------------------------------------
def limpar_nome_modelo(stem: str) -> str:
    """
    Padroniza o nome do modelo para ficar limpo e consistente no CSV e nos gráficos.
    Ajuste se você tiver outros padrões.
    """
    s = stem
    s = s.replace("-pr-", "_").replace("-pr", "").replace("pr_", "")
    return s

def ler_serie_diaria_txt(
    path: Path,
    start: str,
    end: str,
    n_cols_expected: int | None = None,
    ajustar_1_linha_extra: str = "fim",  # "fim" (padrão) ou "inicio"
) -> pd.DataFrame:
    """
    Lê arquivo .txt sem cabeçalho, separado por espaços, e cria índice diário [start, end].

    Robustez:
    - remove linhas totalmente vazias (ex.: newline extra)
    - se houver exatamente 1 linha extra, corta do fim (padrão) ou do início
    """
    df = pd.read_csv(path, sep=r"\s+", header=None, engine="python")

    # 1) remove linhas totalmente vazias (muito comum no fim)
    df = df.dropna(how="all")

    # 2) checa colunas
    if n_cols_expected is not None and df.shape[1] != n_cols_expected:
        raise ValueError(
            f"{path.name}: número de colunas ({df.shape[1]}) difere do esperado ({n_cols_expected})."
        )

    # 3) índice esperado
    idx = pd.date_range(start=start, end=end, freq="D")
    n_esp = len(idx)
    n_obs = len(df)

    # 4) ajusta se tiver 1 linha extra
    if n_obs != n_esp:
        if n_obs == n_esp + 1:
            if ajustar_1_linha_extra.lower() == "inicio":
                df = df.iloc[1:].reset_index(drop=True)   # corta primeira linha
            else:
                df = df.iloc[:-1].reset_index(drop=True)  # corta última linha (padrão)
        else:
            raise ValueError(
                f"{path.name}: número de linhas ({n_obs}) não bate com período {start} a {end} "
                f"({n_esp} dias)."
            )

    df.index = idx
    return df


def diario_para_anual_media(df_diario: pd.DataFrame) -> pd.DataFrame:
    """
    (MÉDIA) Diário -> Anual (média anual)
    Retorna DataFrame anual (uma linha por ano).
    """
    return df_diario.resample("YS").mean()

def nivel_periodo_por_mediana_anual(df_anual: pd.DataFrame, start: str, end: str) -> pd.Series:
    """
    (MEDIANA) Nível do período:
    - Filtra anos do intervalo [start, end]
    - Calcula MEDIANA das médias anuais (por coluna/minibacia)
    """
    y0 = pd.to_datetime(start).year
    y1 = pd.to_datetime(end).year
    sub = df_anual[(df_anual.index.year >= y0) & (df_anual.index.year <= y1)]
    return sub.median(axis=0)

# ---------------------------------------------------
# 1) LER DADOS HISTÓRICOS (BASE) E CALCULAR Q_pres ROBUSTO
# ---------------------------------------------------
print("\n[ETAPA 1] Lendo histórico/base e calculando Q_pres (robusto)...")

# Lê diário base (1980–2023)
obs = ler_serie_diaria_txt(obs_file, OBS_START, OBS_END)

# Nomear colunas como mb_1..mb_N
n_minibacias = obs.shape[1]
obs.columns = [f"mb_{i+1}" for i in range(n_minibacias)]

# (MÉDIA) Converte diário -> série anual (média anual)
obs_anual = diario_para_anual_media(obs)

# (MEDIANA) Calcula Q_pres como mediana das médias anuais no período base
Q_pres = nivel_periodo_por_mediana_anual(obs_anual, ref_start, ref_end)

# Proteção contra Q_pres = 0
Q_pres_safe = Q_pres.copy()
Q_pres_safe[np.abs(Q_pres_safe) < EPS] = np.nan

print(f"  Minibacias: {n_minibacias}")
print(f"  Q_pres calculado como: MEDIANA das MÉDIAS anuais (1980–2023)")

# ---------------------------------------------------
# 2) FUNÇÃO PARA PROCESSAR 1 MODELO E CALCULAR ΔQ/Q
# ---------------------------------------------------
def calcula_delta_q_q_modelo(model_file: Path, Q_pres_safe: pd.Series, horizons: dict) -> pd.DataFrame:
    """
    Para um arquivo de modelo (2015–2100):
      - Lê série diária
      - (MÉDIA) diário -> anual (média anual)
      - (MEDIANA) nível por horizonte = mediana das médias anuais do horizonte
      - ΔQ/Q (%) por minibacia e horizonte
    Retorna DataFrame em formato longo.
    """

    df = ler_serie_diaria_txt(model_file, FUT_START, FUT_END, n_cols_expected=len(Q_pres_safe))
    df.columns = Q_pres_safe.index  # mesmas minibacias do base

    # (MÉDIA) diário -> anual
    df_anual = diario_para_anual_media(df)

    model_name = limpar_nome_modelo(model_file.stem)
    registros = []

    for horiz_name, (h_start, h_end) in horizons.items():
        # (MEDIANA) nível do horizonte = mediana das médias anuais no horizonte
        Q_fut = nivel_periodo_por_mediana_anual(df_anual, h_start, h_end)

        # ΔQ/Q (%) (robusto, porque Q_fut e Q_pres foram calculados por mediana anual)
        delta = (Q_fut - Q_pres_safe) / Q_pres_safe * 100.0

        tmp = pd.DataFrame({
            "modelo": model_name,
            "horizonte": horiz_name,
            "minibacia": delta.index,
            "delta_q_q": delta.values
        })
        registros.append(tmp)

    return pd.concat(registros, ignore_index=True)

# ---------------------------------------------------
# 3) CALCULAR ΔQ/Q PARA TODOS OS MODELOS
# ---------------------------------------------------
print("\n[ETAPA 2] Processando modelos e calculando ΔQ/Q...")

all_results = []
model_files = sorted(base_dir.glob(model_pattern))

if not model_files:
    raise FileNotFoundError(f"Nenhum arquivo encontrado em: {base_dir} com padrão {model_pattern}")

for model_file in model_files:
    if model_file.name.lower() == "observado.txt":
        continue
    print(f"  → {model_file.name}")
    res_model = calcula_delta_q_q_modelo(model_file, Q_pres_safe, horizons)
    all_results.append(res_model)

delta_results = pd.concat(all_results, ignore_index=True)

# Remove linhas com NaN (caso Q_pres tenha sido 0 em alguma minibacia)
delta_results = delta_results.dropna(subset=["delta_q_q"]).copy()

# Salvar CSV
csv_out = output_root / "delta_q_q_por_modelo_minibacia_ssp245.csv"
delta_results.to_csv(csv_out, index=False, encoding="utf-8-sig")
print(f"\n  CSV salvo: {csv_out}")

# ---------------------------------------------------
# 4) BOXPLOTS POR HORIZONTE
# ---------------------------------------------------
print("\n[ETAPA 3] Gerando boxplots por horizonte...")

# Garantir ordem estável dos modelos no eixo X
modelos = sorted(delta_results["modelo"].unique())

for horiz_name in horizons.keys():
    df_h = delta_results[delta_results["horizonte"] == horiz_name]

    # lista de vetores (um por modelo)
    data_box = [df_h.loc[df_h["modelo"] == m, "delta_q_q"].values for m in modelos]

    fig, ax = plt.subplots(figsize=(12, 6), dpi=300)
    ax.boxplot(data_box, labels=modelos, showfliers=False)

    ax.axhline(0, linestyle="--", linewidth=1, color="grey")
    ax.set_title(f"ΔQ/Q por modelo – horizonte {horiz_name}\n"
                 )
    ax.set_xlabel("Modelo climático")
    ax.set_ylabel("ΔQ/Q (%) – distribuição entre minibacias")

    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()

    fig_out = output_root / f"boxplot_delta_q_q_modelos_mediana_anual_{horiz_name}.png"
    plt.savefig(fig_out, dpi=300)
    plt.close(fig)

    print(f"  Figura salva: {fig_out.name}")

print(f"\nConcluído. CSV e figuras salvos em: {output_root}")

In [ ]:
"""
Script: gera_shapefiles_deltaQQ_e_mapas_3paineis.py

Descrição geral
---------------
Este script faz duas etapas principais, em sequência:

1) Geração dos shapefiles temáticos de ΔQ/Q (%)
   - Lê um CSV com resultados de ΔQ/Q por minibacia, modelo e horizonte.
   - Cruza esses dados com o shapefile das minibacias (minis_mgb.shp).
   - Para cada combinação (modelo, horizonte), grava um shapefile do tipo:
       minis_deltaQQ_{modelo_limpo}_{horiz_limpo}.shp
     onde:
       - modelo_limpo  = nome do modelo com "-" e espaços trocados por "_"
       - horiz_limpo   = horizonte com "-" removido e "_" no lugar (ex.: 2015_2040)

2) Geração de mapas 3×1 por modelo (estilo figura final da dissertação)
   - Lê os shapefiles gerados na etapa (1).
   - Calcula uma escala de cores global (vmin, vmax) usando todos os valores de ΔQ/Q (%)
     de todos os modelos e horizontes (1º e 99º percentil, para reduzir outliers extremos).
   - Para cada modelo, gera uma figura com 3 painéis (um por horizonte futuro),
     utilizando a mesma escala de cores em todos.
   - Sobrepõe:
       • contorno das sub-bacias (Subbacias.shp)
       • pontos das cidades de Curitiba e União da Vitória
       • rótulos das cidades com halo (texto branco com borda preta)
   - Salva as figuras no padrão:
       MAPA_3PAINEIS_FIXO_{modelo}.png

Uso esperado na pipeline
------------------------
1. Ajustar os CAMINHOS na seção de CONFIGURAÇÕES abaixo.
2. Executar o script.
3. Usar os shapefiles gerados (shapes_deltaQQ) para outras análises, se necessário.
4. Usar as figuras (figuras_3paineis_FINAL) diretamente na dissertação / artigos.
"""

# ===================================================
# IMPORTAÇÕES
# ===================================================
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import patheffects
from matplotlib.colors import TwoSlopeNorm
from pathlib import Path
import numpy as np

# ===================================================
# 0. CONFIGURAÇÕES – AJUSTAR AQUI
# ===================================================

# Pasta "principal" do cenário (onde está o CSV de ΔQ/Q e onde você quer guardar os outputs)
base_dir = Path(r"E:\RESULTADOS\SSP5_85")  # AJUSTE AQUI SE PRECISAR

# CSV gerado pelo script anterior com ΔQ/Q por minibacia, modelo e horizonte
delta_csv = base_dir / "BoxPlot_Modelos\delta_q_q_por_modelo_minibacia_ssp585.csv"

# Shapefile das minibacias MGB
shp_minis = Path(r"E:\IGUAÇU_OTTO\6_Calibração\minis_mgb.shp")
ID_FIELD  = "ID_Mini"  # campo de ID da minibacia no shapefile

# Pasta onde serão salvos os shapefiles temáticos (saída da ETAPA 1)
shapes_dir = base_dir / "shapes_deltaQQ_Mediana"
shapes_dir.mkdir(exist_ok=True)

# Pasta onde serão salvos os mapas 3×1 finais (saída da ETAPA 2)
fig_out_dir = base_dir / "Mapas_3paineis_Mediana"
fig_out_dir.mkdir(exist_ok=True)

# Shapefile das sub-bacias (para contorno)
shp_sub = Path(r"E:\IGUAÇU_OTTO\Shp\Subbacias.shp")

# Shapefile das cidades (contendo Curitiba e União da Vitória)
shp_cidades = Path(
    r"G:\Meu Drive\2_MESTRADO\1_Dissertação\Figuras\20250516_SHAPES_FIGURA\GEOFT_CIDADE_2016.shp"
)

# Campo com o nome da cidade no shapefile de cidades
CAMPO_NOME_CIDADE = "CID_NM"

# ===================================================
# 1. ETAPA 1 – GERAR SHAPEFILES TEMÁTICOS A PARTIR DO CSV
# ===================================================

print("\n=== ETAPA 1: Gerando shapefiles de ΔQ/Q por modelo e horizonte ===\n")

# 1.1 Ler CSV com ΔQ/Q
delta_results = pd.read_csv(delta_csv)

# Esperado: colunas ['modelo', 'horizonte', 'minibacia', 'delta_q_q']
# Transformar "mb_1", "mb_2", ... → 1, 2, ...
delta_results["mini"] = (
    delta_results["minibacia"]
    .str.replace("mb_", "", regex=False)
    .astype(int)
)

# 1.2 Ler shapefile das minibacias e garantir tipo inteiro no campo de ID
minis_gdf = gpd.read_file(shp_minis)
minis_gdf[ID_FIELD] = minis_gdf[ID_FIELD].astype(int)

# 1.3 Listar modelos e horizontes disponíveis no CSV
# Remover "-pr-" dos nomes antes de qualquer processamento
delta_results["modelo"] = (
    delta_results["modelo"]
    .str.replace("-pr-", "_", regex=False)     # troca -pr- por _
    .str.replace("-pr",  "", regex=False)      # remove -pr se estiver no final
    .str.replace("pr_",  "", regex=False)      # remove pr_ se tiver vindo de outra forma
)
modelos = sorted(delta_results["modelo"].unique())
horizontes = sorted(delta_results["horizonte"].unique())

print(f"Modelos encontrados no CSV ({len(modelos)}):")
for m in modelos:
    print("  →", m)

print("\nHorizontes encontrados no CSV:")
for h in horizontes:
    print("  →", h)

# 1.4 Loop: para cada combinação (modelo, horizonte), gerar shapefile
for modelo in modelos:
    for horiz in horizontes:
        df_mh = delta_results[
            (delta_results["modelo"] == modelo) &
            (delta_results["horizonte"] == horiz)
        ][["mini", "delta_q_q"]].copy()

        # Merge com as minibacias
        gdf_mh = minis_gdf.merge(df_mh, left_on=ID_FIELD, right_on="mini", how="left")

        # Limpar nomes para usar no arquivo
        modelo_limpo = modelo.replace("-", "_").replace(" ", "_")
        horiz_limpo  = horiz.replace(" ", "").replace("-", "_")  # ex.: "2015-2040" → "2015_2040"

        shp_out = shapes_dir / f"minis_deltaQQ_{modelo_limpo}_{horiz_limpo}.shp"
        gdf_mh.to_file(shp_out)

        print(f"Shapefile salvo: {shp_out.name}")

print("\n✔ ETAPA 1 concluída: shapefiles temáticos gerados em:", shapes_dir)

# ===================================================
# 2. ETAPA 2 – GERAR MAPAS 3×1 (ESTILO CÓDIGO 3)
# ===================================================

print("\n=== ETAPA 2: Gerando mapas 3×1 por modelo (com sub-bacias e cidades) ===\n")

# 2.1 Definir rótulos e códigos dos horizontes (fixos)
horiz_labels = ["2015-2040", "2041-2070", "2071-2100"]
horiz_codes  = ["2015_2040", "2041_2070", "2071_2100"]

print("Horizontes usados na ETAPA 2:")
for label, code in zip(horiz_labels, horiz_codes):
    print(f"  {label}  →  {code}")

# 2.2 Montar um dicionário {modelo_base: {code: caminho_shp}}
shp_map = {}  # ex.: {"ACCESS_CM2_ssp245": {"2015_2040": Path(...), ...}}

for f in shapes_dir.glob("minis_deltaQQ_*.shp"):
    stem = f.stem  # ex.: "minis_deltaQQ_ACCESS_CM2_ssp245_2015_2040"
    prefix = "minis_deltaQQ_"
    if not stem.startswith(prefix):
        print(f"[AVISO] Nome inesperado (ignorado): {f.name}")
        continue

    resto = stem[len(prefix):]  # tira "minis_deltaQQ_"

    # descobrir qual código de horizonte está no final
    found_code = None
    for code in horiz_codes:
        suffix = "_" + code
        if resto.endswith(suffix):
            found_code = code
            base_model = resto[:-len(suffix)]  # tudo antes de "_2015_2040", por ex.
            break

    if found_code is None:
        print(f"[AVISO] Não reconheci horizonte no nome (ignorado): {f.name}")
        continue

    # registra no dicionário
    if base_model not in shp_map:
        shp_map[base_model] = {}
    shp_map[base_model][found_code] = f

# lista de modelos base
modelos_para_mapear = sorted(shp_map.keys())

print(f"\nModelos base detectados ({len(modelos_para_mapear)}):")
for m in modelos_para_mapear:
    cods_disp = ", ".join(sorted(shp_map[m].keys()))
    print(f"  → {m}  (horizontes: {cods_disp})")

# 2.3 Definir escala global (vmin, vmax) com base em TODOS os shapefiles
valores = []
for base_model, dict_horiz in shp_map.items():
    for code, shp_path in dict_horiz.items():
        df_tmp = gpd.read_file(shp_path)
        if "delta_q_q" in df_tmp.columns:
            valores.extend(df_tmp["delta_q_q"].dropna().tolist())
        else:
            print(f"[AVISO] 'delta_q_q' não encontrada em {shp_path.name}")

if not valores:
    raise RuntimeError("Não foram encontrados valores de 'delta_q_q' nos shapefiles.")

val_min, val_max = min(valores), max(valores)
print(f"VALORES REAIS → min = {val_min:.1f}  max = {val_max:.1f}")

# usa percentis pra limpar extremos, mas deixa a escala SIMÉTRICA em torno de 0
p1, p99 = np.percentile(valores, [1, 99])
max_abs = max(abs(p1), abs(p99))
vmin, vmax = -max_abs, max_abs

print(f"\nESCALA DIVERGENTE SIMÉTRICA APLICADA → {vmin:.1f}  a  {vmax:.1f}\n")

# normalização divergente centrada em 0
divnorm = TwoSlopeNorm(vmin=vmin, vcenter=0.0, vmax=vmax)

# 2.4 Ler sub-bacias e cidades
gdf_sub = gpd.read_file(shp_sub)
gdf_cid = gpd.read_file(shp_cidades)

print("CRS das CIDADES:", gdf_cid.crs)
print("Campos disponíveis nas CIDADES:", list(gdf_cid.columns))

# Filtro robusto por substring para Curitiba e União da Vitória
mask_cur = gdf_cid[CAMPO_NOME_CIDADE].str.contains("curit", case=False, na=False)
mask_un  = gdf_cid[CAMPO_NOME_CIDADE].str.contains("uni[aã]o da vit", case=False, na=False, regex=True)

cidades_sel = gdf_cid[mask_cur | mask_un].copy()

print("\nCidades selecionadas para plotagem:")
print(cidades_sel[[CAMPO_NOME_CIDADE]].drop_duplicates())

# 2.5 Gerar mapas 3×1 para cada modelo base
for base_model in modelos_para_mapear:

    # garante que o modelo base tenha os 3 horizontes
    if not all(code in shp_map[base_model] for code in horiz_codes):
        print(f"[AVISO] Modelo {base_model} não possui todos os horizontes. Pulando.")
        continue

    print(f"\nGerando figura para o modelo → {base_model}")

    # ler shapefiles das minibacias para cada horizonte na ordem certa
    gdfs = [gpd.read_file(shp_map[base_model][code]) for code in horiz_codes]

    # usar o CRS do primeiro shapefile de minibacias como referência
    crs_minis = gdfs[0].crs

    # reprojetar sub-bacias e cidades para o mesmo CRS das minibacias
    gdf_sub_proj = gdf_sub.to_crs(crs_minis)
    cidades_proj = cidades_sel.to_crs(crs_minis) if not cidades_sel.empty else cidades_sel

    xmin, ymin, xmax, ymax = gdfs[0].total_bounds
    dx = (xmax - xmin) * 0.01  # deslocamento relativo em x
    dy = (ymax - ymin) * 0.01  # deslocamento relativo em y

    fig = plt.figure(figsize=(18, 6), dpi=300)
    gs  = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.05])

    axes = [fig.add_subplot(gs[i]) for i in range(3)]

    for ax, gdf, label in zip(axes, gdfs, horiz_labels):

        # Mapa base das minibacias com ΔQ/Q
        gdf.plot(
            column="delta_q_q",
            cmap="RdYlBu",   # negativo = vermelho, positivo = azul
            norm=divnorm,    # centraliza o 0 na cor neutra
            ax=ax,
            edgecolor="black",
            linewidth=0.08,
        )

        # Contorno das sub-bacias
        gdf_sub_proj.boundary.plot(
            ax=ax,
            edgecolor="grey",
            linewidth=1.2,
            zorder=3
        )

        # Pontos das cidades (se houver)
        if not cidades_proj.empty:
            cidades_proj.plot(
                ax=ax,
                marker="^",
                color="black",
                markersize=40,
                zorder=4,
                linewidth=0
            )

            # Rótulos das cidades com halo
            for _, row in cidades_proj.iterrows():
                x = row.geometry.x
                y = row.geometry.y
                nome = row[CAMPO_NOME_CIDADE]

                txt = ax.text(
                    x + dx,
                    y + dy,
                    nome,
                    fontsize=9,
                    color="white",
                    ha="left",
                    va="bottom",
                    zorder=5,
                )
                txt.set_path_effects([
                    patheffects.Stroke(linewidth=1.5, foreground="black"),
                    patheffects.Normal()
                ])

        ax.set_title(f"Horizonte {label}", fontsize=11)
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal")
        ax.set_axis_off()

    # COLORBAR na coluna 4
    cax = fig.add_subplot(gs[3])
    sm  = plt.cm.ScalarMappable(
        cmap="RdYlBu",
        norm=divnorm
    )
    sm._A = []
    cbar = fig.colorbar(sm, cax=cax)
    cbar.set_label("ΔQ/Q (%)", fontsize=11)

    # Título geral
    fig.suptitle(
        f"ΔQ/Q (%) – Modelo {base_model}",
        fontsize=14,
        weight="bold",
        y=0.97
    )

    # Ajuste fino de margens
    fig.subplots_adjust(left=0.02, right=0.96, top=0.90, bottom=0.03, wspace=0.02)

    # Salvar figura
    fig_path = fig_out_dir / f"MAPA_{base_model}.png"
    fig.savefig(fig_path)
    plt.close(fig)

    print(f"Figura salva: {fig_path.name}")

print("\n✨ ETAPA 2 FINALIZADA – Mapas 3×1 gerados com sucesso.")
print("  → Figuras 3×1:", fig_out_dir)


In [ ]:
"""
===============================================================================
Cálculo de ΔQ/Q (%) a partir do ENSEMBLE (média multimodelo) por minibacia
===============================================================================

Objetivo
--------
Calcular a variação relativa ΔQ/Q (%) entre um período base (1980–2023) e
horizontes futuros (2015–2040, 2041–2070, 2071–2100), usando o ENSEMBLE
multimodelo como série representativa do futuro.

Metodologia (coerente com "vazão média anual" + robustez a extremos)
--------------------------------------------------------------------
Este script segue a mesma lógica adotada na dissertação para reduzir a influência
de anos extremos, sem distorcer o conceito de "vazão média anual":

1) Série anual (TENDÊNCIA/representação física)
   - (MÉDIA) Diário -> ANUAL por mean:
       Q_ano = média diária dentro do ano

2) Nível típico do período (ROBUSTEZ a extremos)
   - (MEDIANA) Para cada período (base/horizonte), define-se o "nível" como:
       Q_periodo = mediana dos valores anuais (médias anuais) do período

3) Ensemble
   - Para cada modelo: gera série ANUAL (média anual).
   - (MÉDIA) Ensemble anual = média aritmética das séries anuais dos modelos.
   - Para cada horizonte: Q_fut_ens = mediana dos valores anuais do ensemble no horizonte.

4) ΔQ/Q (%)
   - ΔQ/Q = (Q_fut_ens - Q_pres) / Q_pres × 100

Por que NÃO usar "mediana diária" diretamente?
----------------------------------------------
A mediana diária ao longo de décadas representa um "dia típico" do período,
e não uma vazão média anual. Para manter coerência hidrológica e robustez,
usa-se:
- média anual para construir a série
- mediana entre anos para resumir o período

Entradas esperadas
------------------
- Observado/base (`Observado.txt`):
  - txt separado por espaço, sem cabeçalho
  - linhas diárias 1980-01-01 a 2023-12-31
  - colunas = minibacias (1..N) na mesma ordem dos modelos

- Modelos (`*-ssp245.txt`, etc.):
  - txt separado por espaço, sem cabeçalho
  - linhas diárias 2015-01-01 a 2100-12-31
  - colunas = minibacias (mesma ordem do observado)

Saídas
------
1) CSV resumo geral:
   - delta_q_q_ensemble_por_minibacia.csv
     colunas: ['minibacia', 'horizonte', 'Q_pres', 'Q_fut_ens', 'delta_q_q_ensemble']

2) CSVs individuais por minibacia:
   - Ensemble_Modelos/Ensemble_Minibacias/
     ex.: ensemble_delta_q_q_mb_1.csv, ...

Dependências
------------
- pandas
- numpy
- pathlib
===============================================================================
"""

import numpy as np
import pandas as pd
from pathlib import Path

# ---------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------
root_dir = Path(r"E:\RESULTADOS_AB2\SSP2-45")
base_dir = root_dir / "QTudo_Fut_Cenario 2"
obs_file = root_dir / "Qtudo_Pres" / "Observado.txt"

model_pattern = "*-ssp245.txt"

# Período base
ref_start = "1980-01-01"
ref_end   = "2023-12-31"

# Horizontes futuros
horizons = {
    "2015-2040": ("2015-01-01", "2040-12-31"),
    "2041-2070": ("2041-01-01", "2070-12-31"),
    "2071-2100": ("2071-01-01", "2100-12-31"),
    "2015-2100": ("2015-01-01", "2100-12-31"),
}

# Datas completas esperadas
OBS_START, OBS_END = "1980-01-01", "2023-12-31"
FUT_START, FUT_END = "2015-01-01", "2100-12-31"

# Saídas
output_root = root_dir / "Ensemble_Modelos"
output_root.mkdir(parents=True, exist_ok=True)

csv_out_resumo = output_root / "delta_q_q_ensemble_por_minibacia.csv"

out_minibacias_dir = output_root / "Ensemble_Minibacias"
out_minibacias_dir.mkdir(parents=True, exist_ok=True)

# Proteção contra divisão por zero
EPS = 1e-12

# ---------------------------------------------------
# FUNÇÕES
# ---------------------------------------------------
def ler_txt_diario(
    path: Path,
    start: str,
    end: str,
    n_cols_expected: int | None = None,
    ajustar_1_linha_extra: str = "fim",  # "fim" (padrão) ou "inicio"
) -> pd.DataFrame:
    """
    Lê TXT diário (espaço, sem header) e atribui índice diário.

    Robustez:
    - remove linhas totalmente vazias (ex.: newline/linha em branco no final)
    - se houver exatamente 1 linha extra, corta do fim (padrão) ou do início
    """
    df = pd.read_csv(path, sep=r"\s+", header=None, engine="python")

    # 1) remove linhas totalmente vazias
    df = df.dropna(how="all")

    # 2) checa colunas
    if n_cols_expected is not None and df.shape[1] != n_cols_expected:
        raise ValueError(f"{path.name}: colunas {df.shape[1]} != esperado {n_cols_expected}")

    # 3) índice esperado
    idx = pd.date_range(start=start, end=end, freq="D")
    n_esp = len(idx)
    n_obs = len(df)

    # 4) ajusta diferença de 1 linha (caso típico)
    if n_obs != n_esp:
        if n_obs == n_esp + 1:
            if ajustar_1_linha_extra.lower() == "inicio":
                df = df.iloc[1:].reset_index(drop=True)   # corta 1ª linha
            else:
                df = df.iloc[:-1].reset_index(drop=True)  # corta última linha (padrão)
        else:
            raise ValueError(f"{path.name}: linhas {n_obs} != dias {n_esp} ({start} a {end})")

    df.index = idx
    return df


def diario_para_anual_media(df_diario: pd.DataFrame) -> pd.DataFrame:
    """(MÉDIA) diário -> anual (média anual)."""
    return df_diario.resample("YS").mean()

def nivel_periodo_mediana_anual(df_anual: pd.DataFrame, start: str, end: str) -> pd.Series:
    """(MEDIANA) nível do período = mediana das médias anuais dentro do período."""
    y0 = pd.to_datetime(start).year
    y1 = pd.to_datetime(end).year
    sub = df_anual[(df_anual.index.year >= y0) & (df_anual.index.year <= y1)]
    return sub.median(axis=0)

# ---------------------------------------------------
# ETAPA 1: LER BASE E CALCULAR Q_pres (ROBUSTO)
# ---------------------------------------------------
print("\n=== ETAPA 1: Lendo base e calculando Q_pres (mediana das médias anuais) ===")

obs = ler_txt_diario(obs_file, OBS_START, OBS_END)

n_minibacias = obs.shape[1]
obs.columns = [f"mb_{i+1}" for i in range(n_minibacias)]

# (MÉDIA) diário -> anual
obs_anual = diario_para_anual_media(obs)

# (MEDIANA) nível do período base
Q_pres = nivel_periodo_mediana_anual(obs_anual, ref_start, ref_end)

# Proteção contra Q_pres ~ 0
Q_pres_safe = Q_pres.copy()
Q_pres_safe[np.abs(Q_pres_safe) < EPS] = np.nan

print(f"Minibacias detectadas: {len(Q_pres_safe)}")
print("Exemplo Q_pres (5 primeiras):")
print(Q_pres_safe.head())

# ---------------------------------------------------
# ETAPA 2: LER MODELOS E CONSTRUIR ENSEMBLE ANUAL
# ---------------------------------------------------
print("\n=== ETAPA 2: Lendo modelos e construindo ENSEMBLE ANUAL (média entre modelos) ===")

model_files = sorted([f for f in base_dir.glob(model_pattern) if f.name.lower() != "observado.txt"])
if not model_files:
    raise RuntimeError(f"Nenhum modelo encontrado em {base_dir} com padrão {model_pattern}")

print(f"Modelos encontrados: {len(model_files)}")
for mf in model_files:
    print("  →", mf.name)

# Acumulador do ensemble anual
ensemble_anual_soma = None
n_models = 0

# Eixo anual do futuro (para garantir alinhamento)
anos_fut = pd.date_range(start=FUT_START, end=FUT_END, freq="YS")

for i, model_file in enumerate(model_files, start=1):
    print(f"\nLendo modelo {i}/{len(model_files)}: {model_file.name}")

    df = ler_txt_diario(model_file, FUT_START, FUT_END, n_cols_expected=len(Q_pres_safe))
    df.columns = Q_pres_safe.index

    # (MÉDIA) diário -> anual
    df_anual = diario_para_anual_media(df).reindex(anos_fut)

    if df_anual.isna().any().any():
        raise ValueError(f"{model_file.name}: NaNs após reindex anual. Verifique integridade temporal.")

    if ensemble_anual_soma is None:
        ensemble_anual_soma = df_anual.astype(np.float64).copy()
    else:
        ensemble_anual_soma += df_anual.astype(np.float64)

    n_models += 1

# (MÉDIA) ensemble anual = média entre modelos
ensemble_anual = ensemble_anual_soma / float(n_models)

print(f"\nEnsemble anual calculado com {n_models} modelos.")
print("Dimensões:", ensemble_anual.shape)
print("Período:", ensemble_anual.index.min(), "a", ensemble_anual.index.max())

# ---------------------------------------------------
# ETAPA 3: CALCULAR Q_fut_ens (ROBUSTO) E ΔQ/Q
# ---------------------------------------------------
print("\n=== ETAPA 3: Calculando Q_fut_ens (mediana anual) e ΔQ/Q (%) ===")

registros = []

for horiz_name, (h_start, h_end) in horizons.items():
    print(f"  Horizonte {horiz_name}: {h_start} a {h_end}")

    # (MEDIANA) nível do horizonte no ensemble anual
    Q_fut_ens = nivel_periodo_mediana_anual(ensemble_anual, h_start, h_end)

    # ΔQ/Q (%) com proteção contra zero
    delta = (Q_fut_ens - Q_pres_safe) / Q_pres_safe * 100.0

    tmp = pd.DataFrame({
        "minibacia": Q_pres_safe.index,
        "horizonte": horiz_name,
        "Q_pres": Q_pres_safe.values,
        "Q_fut_ens": Q_fut_ens.values,
        "delta_q_q_ensemble": delta.values
    })

    registros.append(tmp)

df_result = pd.concat(registros, ignore_index=True)
df_result = df_result.sort_values(["minibacia", "horizonte"])

# Remover NaNs (caso Q_pres seja zero ou faltante)
df_result = df_result.dropna(subset=["delta_q_q_ensemble"]).copy()

df_result.to_csv(csv_out_resumo, index=False, encoding="utf-8-sig")
print(f"\nCSV resumo salvo em: {csv_out_resumo}")

# ---------------------------------------------------
# ETAPA 4: GERAR CSV POR MINIBACIA (OPCIONAL)
# ---------------------------------------------------
print("\n=== ETAPA 4: Gerando CSV individual por minibacia (opcional) ===")

minibacias = df_result["minibacia"].unique()
print(f"Minibacias para exportar: {len(minibacias)}")

for mb in minibacias:
    df_mb = df_result[df_result["minibacia"] == mb].copy()
    out_path = out_minibacias_dir / f"ensemble_delta_q_q_{mb}.csv"
    df_mb.to_csv(out_path, index=False, encoding="utf-8-sig")

print("\n✅ Processo concluído com sucesso.")
print(f"  → CSV resumo: {csv_out_resumo}")
print(f"  → CSVs por minibacia em: {out_minibacias_dir}")

In [ ]:
"""
Script: gera_shapefile_e_mapas_ensemble_3e4paineis.py

O que este script faz
---------------------
1) Gera shapefiles temáticos de ΔQ/Q do ensemble por horizonte, a partir do CSV:
   delta_q_q_ensemble_por_minibacia.csv

2) Gera dois produtos:
   A) Mapa 3×1 (3 painéis): 2015–2040, 2041–2070, 2071–2100
   B) Mapa 2×2 (4 painéis): Curto, Médio, Longo e Total (2015–2100)

A escala é fixa em [-100, +100] com saturação indicada na colorbar:
"< -100" e "> 100".
"""


# ===================================================
# IMPORTAÇÕES
# ===================================================
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import patheffects
from matplotlib.colors import TwoSlopeNorm
from pathlib import Path

# ===================================================
# 0. CONFIGURAÇÕES – AJUSTAR AQUI
# ===================================================

# Pasta raiz do cenário
base_dir = Path(r"E:\RESULTADOS_AB2\SSP2-45")

# CSV com ΔQ/Q ensemble por minibacia e horizonte
ensemble_csv = base_dir / r"Ensemble_Modelos\delta_q_q_ensemble_por_minibacia.csv"

# Shapefile das minibacias MGB
shp_minis = Path(r"E:\IGUAÇU_OTTO\6_Calibração\minis_mgb.shp")
ID_FIELD  = "ID_Mini"  # campo de ID da minibacia no shapefile

# Pasta para shapefiles do ensemble
shapes_dir = base_dir / "shapes_deltaQQ_ensemble"
shapes_dir.mkdir(exist_ok=True)

# Pasta para figuras
fig_out_dir = base_dir / "figuras_ensemble"
fig_out_dir.mkdir(exist_ok=True)

# Shapefile das sub-bacias (para contorno)
shp_sub = Path(r"E:\IGUAÇU_OTTO\Shp\Subbacias.shp")

# Shapefile das cidades (contendo Curitiba e União da Vitória)
shp_cidades = Path(
    r"G:\Meu Drive\2_MESTRADO\1_Dissertação\Figuras\20250516_SHAPES_FIGURA\GEOFT_CIDADE_2016.shp"
)

# Campo com o nome da cidade no shapefile de cidades
CAMPO_NOME_CIDADE = "CID_NM"

# ---------------------------------------------------
# Horizontes: 3 painéis e 4 painéis (inclui Total)
# ---------------------------------------------------
horiz_labels_3 = ["2015-2040", "2041-2070", "2071-2100"]
horiz_codes_3  = ["2015_2040", "2041_2070", "2071_2100"]

horiz_labels_4 = ["2015-2040", "2041-2070", "2071-2100", "2015-2100"]
horiz_codes_4  = ["2015_2040", "2041_2070", "2071_2100", "2015_2100"]

# Rótulos mais didáticos (opcional)
rotulos_paineis = {
    "2015-2040": "Horizonte 2015–2040",
    "2041-2070": "Horizonte 2041–2070",
    "2071-2100": "Horizonte 2071–2100",
    "2015-2100": "Horizonte Total (2015–2100)",
}

# ---------------------------------------------------
# Escala fixa
# ---------------------------------------------------
vmin, vmax = -100.0, 100.0
divnorm = TwoSlopeNorm(vmin=vmin, vcenter=0.0, vmax=vmax)

# ===================================================
# 1. ETAPA 1 – GERAR SHAPEFILES DE ΔQ/Q ENSEMBLE
# ===================================================

print("\n=== ETAPA 1: Gerando shapefiles de ΔQ/Q ensemble por horizonte ===\n")

df_ens = pd.read_csv(ensemble_csv)

col_esp = {"minibacia", "horizonte", "delta_q_q_ensemble"}
if not col_esp.issubset(df_ens.columns):
    raise ValueError(
        f"O CSV de ensemble não contém as colunas esperadas. "
        f"Esperado pelo menos: {col_esp}, encontrado: {set(df_ens.columns)}"
    )

# "mb_1" → 1
df_ens["mini"] = df_ens["minibacia"].str.replace("mb_", "", regex=False).astype(int)

minis_gdf = gpd.read_file(shp_minis)
minis_gdf[ID_FIELD] = minis_gdf[ID_FIELD].astype(int)

# Gerar shapefiles para TODOS os horizontes que vamos usar (3 + total)
for h_label, h_code in zip(horiz_labels_4, horiz_codes_4):
    print(f"  Horizonte {h_label} → código {h_code}")

    df_h = df_ens[df_ens["horizonte"] == h_label].copy()
    if df_h.empty:
        print(f"   [AVISO] Não há dados para o horizonte {h_label} no CSV. Pulando.")
        continue

    gdf_h = minis_gdf.merge(
        df_h[["mini", "delta_q_q_ensemble"]],
        left_on=ID_FIELD,
        right_on="mini",
        how="left"
    )

    gdf_h = gdf_h.rename(columns={"delta_q_q_ensemble": "delta_q_q"})

    shp_out = shapes_dir / f"minis_deltaQQ_ensemble_{h_code}.shp"
    gdf_h.to_file(shp_out)
    print(f"   Shapefile ensemble salvo: {shp_out.name}")

print("\n✔ ETAPA 1 concluída: shapefiles ensemble gerados em:", shapes_dir)

# ===================================================
# 2. ETAPA 2 – FUNÇÕES DE MAPA (3×1 e 2×2)
# ===================================================

print("\n=== ETAPA 2: Gerando mapas do ensemble (3 painéis e 4 painéis) ===\n")

def carregar_gdfs(codes: list[str]) -> list[gpd.GeoDataFrame]:
    gdfs_local = []
    for code in codes:
        shp_path = shapes_dir / f"minis_deltaQQ_ensemble_{code}.shp"
        if not shp_path.exists():
            raise FileNotFoundError(f"Shapefile não encontrado: {shp_path}")
        gdfs_local.append(gpd.read_file(shp_path))
    return gdfs_local

def preparar_camadas(crs_minis):
    gdf_sub = gpd.read_file(shp_sub).to_crs(crs_minis)
    gdf_cid = gpd.read_file(shp_cidades).to_crs(crs_minis)

    mask_cur = gdf_cid[CAMPO_NOME_CIDADE].str.contains(r"^curitiba$", case=False, na=False, regex=True)
    mask_un  = gdf_cid[CAMPO_NOME_CIDADE].str.contains("uni[aã]o da vit", case=False, na=False, regex=True)
    cidades = gdf_cid[mask_cur | mask_un].copy()

    return gdf_sub, cidades

def aplicar_colorbar(fig, cax):
    sm = plt.cm.ScalarMappable(cmap="RdYlBu", norm=divnorm)
    sm._A = []
    cbar = fig.colorbar(sm, cax=cax)
    cbar.set_label("ΔQ/Q (%) – mediana anual do ensemble", fontsize=11)

    ticks = [-100, -75, -50, -25, 0, 25, 50, 75, 100]
    labels = ["< -100", "-75", "-50", "-25", "0", "25", "50", "75", "> 100"]
    cbar.set_ticks(ticks)
    cbar.set_ticklabels(labels)

def plot_painel(ax, gdf, title, gdf_sub, cidades, bounds):
    xmin, ymin, xmax, ymax = bounds
    dx = (xmax - xmin) * 0.01
    dy = (ymax - ymin) * 0.01

    gdf.plot(
        column="delta_q_q",
        cmap="RdYlBu",
        norm=divnorm,
        ax=ax,
        edgecolor="black",
        linewidth=0.08,
    )

    gdf_sub.boundary.plot(ax=ax, edgecolor="grey", linewidth=1.2, zorder=3)

    if not cidades.empty:
        cidades.plot(ax=ax, marker="^", color="black", markersize=40, zorder=4, linewidth=0)
        for _, row in cidades.iterrows():
            x = row.geometry.x
            y = row.geometry.y
            nome = row[CAMPO_NOME_CIDADE]
            txt = ax.text(
                x + dx, y + dy, nome,
                fontsize=9, color="white",
                ha="left", va="bottom", zorder=5
            )
            txt.set_path_effects([
                patheffects.Stroke(linewidth=1.5, foreground="black"),
                patheffects.Normal()
            ])

    ax.set_title(title, fontsize=11, pad=5)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_aspect("equal")
    ax.set_axis_off()

# ===================================================
# 3. MAPA 3×1
# ===================================================

gdfs_3 = carregar_gdfs(horiz_codes_3)
crs_minis = gdfs_3[0].crs
gdf_sub_proj, cidades_proj = preparar_camadas(crs_minis)
bounds = gdfs_3[0].total_bounds

fig = plt.figure(figsize=(14, 6), dpi=300)
gs  = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.06])

axes = [fig.add_subplot(gs[i]) for i in range(3)]

for ax, gdf, label in zip(axes, gdfs_3, horiz_labels_3):
    plot_painel(ax, gdf, rotulos_paineis[label], gdf_sub_proj, cidades_proj, bounds)

cax = fig.add_subplot(gs[3])
aplicar_colorbar(fig, cax)

fig.suptitle("ΔQ/Q (%) – Ensemble (SSP2-4.5)", fontsize=14, weight="bold", y=0.97)
fig.subplots_adjust(left=0.02, right=0.95, top=0.90, bottom=0.03, wspace=0.02)

fig_path_3 = fig_out_dir / "MAPA_3PAINEIS_ENSEMBLE.png"
fig.savefig(fig_path_3, dpi=300)
plt.close(fig)

print("✔ Figura 3×1 salva:", fig_path_3)

# ===================================================
# 4. MAPA 2×2 (4 PAINÉIS, INCLUI TOTAL)
# ===================================================

# Se o CSV não tiver "2015-2100", esta parte vai falhar ao carregar o shapefile.
gdfs_4 = carregar_gdfs(horiz_codes_4)
crs_minis = gdfs_4[0].crs
gdf_sub_proj, cidades_proj = preparar_camadas(crs_minis)
bounds = gdfs_4[0].total_bounds

fig = plt.figure(figsize=(16, 10), dpi=300)
gs = gridspec.GridSpec(  2, 3,  width_ratios=[1, 1, 0.05],  # colorbar bem mais estreita
    wspace=0.04,   hspace=0.08)


ax_11 = fig.add_subplot(gs[0, 0])
ax_12 = fig.add_subplot(gs[0, 1])
ax_21 = fig.add_subplot(gs[1, 0])
ax_22 = fig.add_subplot(gs[1, 1])
cax   = fig.add_subplot(gs[:, 2])

axes = [ax_11, ax_12, ax_21, ax_22]

for ax, gdf, label in zip(axes, gdfs_4, horiz_labels_4):
    plot_painel(ax, gdf, rotulos_paineis[label], gdf_sub_proj, cidades_proj, bounds)

aplicar_colorbar(fig, cax)

fig.suptitle("ΔQ/Q (%) – Ensemble (SSP2-4.5)", fontsize=14, weight="bold", y=0.98)
fig.tight_layout(rect=[0.03, 0.03, 0.97, 0.95])

fig_path_4 = fig_out_dir / "MAPA_4PAINEIS_ENSEMBLE_2.png"
fig.savefig(fig_path_4, dpi=300)
plt.close(fig)

print("✔ Figura 2×2 salva:", fig_path_4)
print("\n✨ Concluído: mapas 3×1 e 4 painéis gerados.")

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from pathlib import Path
import pandas as pd
import numpy as np

# ==============================
# CAMINHOS
# ==============================
base_dir = Path(r"C:\Users\Matheus Marinho\Downloads\Nova pasta\Leitor de Binários\shapes_deltaQQ")

# horizontes
horiz_codes  = ["2015_2040", "2041_2070", "2071_2100"]
horiz_labels = ["2015-2040", "2041-2070", "2071-2100"]

# saída
out = base_dir.parent / "figuras_3painéis_FINAL"
out.mkdir(exist_ok=True)

# ==============================
# 1. LISTAR MODELOS
# ==============================
modelos = sorted({ "_".join(f.stem.split("_")[2:-2]) for f in base_dir.glob("minis_deltaQQ_*.shp") })

print(f"\nModelos detectados ({len(modelos)}):")
for m in modelos: print(" →", m)

# ==============================
# 2. DEFINIR ESCALA GLOBAL P/ TODOS MODELOS
# ==============================
valores = []
for modelo in modelos:
    for code in horiz_codes:
        df = gpd.read_file(base_dir / f"minis_deltaQQ_{modelo}_{code}.shp")
        valores.extend(df["delta_q_q"].tolist())

# escala fixa global
vmin, vmax = np.percentile(valores, [1, 99])   # limpa outliers extremos
# OU se quiser usar o range absoluto real → comente acima e descomente abaixo
# vmin, vmax = min(valores), max(valores)

print(f"\nESCALA GLOBAL FIXA APLICADA A TODOS MAPAS → {vmin:.1f}  a  {vmax:.1f}\n")

# ==============================
# 3. MAPA 3×1 PARA CADA MODELO (SEM DISTORÇÃO)
# ==============================
for modelo in modelos:

    print(f"\nGerando figura → {modelo}")

    gdfs = [gpd.read_file(base_dir / f"minis_deltaQQ_{modelo}_{code}.shp")
            for code in horiz_codes]

    xmin, ymin, xmax, ymax = gdfs[0].total_bounds

    # Grade 1×3 + barra lateral fixa (coluna 4)
    fig = plt.figure(figsize=(18,6), dpi=300)
    gs  = gridspec.GridSpec(1,4, width_ratios=[1,1,1,0.05])

    axes = [fig.add_subplot(gs[i]) for i in range(3)]

    for ax, gdf, label in zip(axes, gdfs, horiz_labels):

        gdf.plot(column="delta_q_q",
                 cmap="RdYlBu_r",
                 ax=ax,
                 vmin=vmin, vmax=vmax,
                 edgecolor="black", linewidth=0.08)

        ax.set_title(f"Horizonte {label}", fontsize=11)
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal")
        ax.set_axis_off()

    # COLORBAR na coluna 4 (sem encolher o 3º painel)
    cax = fig.add_subplot(gs[3])
    sm  = plt.cm.ScalarMappable(cmap="RdYlBu_r", norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm._A = []
    cbar = fig.colorbar(sm, cax=cax)
    cbar.set_label("ΔQ/Q (%)", fontsize=11)

    fig.suptitle(f"ΔQ/Q (%) – Modelo {modelo}", fontsize=14, weight="bold", y=1.02)
    fig.tight_layout()

    fig.savefig(out / f"MAPA_3PAINEIS_FIXO_{modelo}.png", bbox_inches="tight")
    plt.close(fig)

print("\n✨ FINALIZADO – Todos os mapas gerados com escala fixa e painéis alinhados.")
print(f"Pasta de saída: {out}")

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import patheffects
from pathlib import Path
import numpy as np

# ==============================
# CAMINHOS
# ==============================
base_dir = Path(r"C:\Users\Matheus Marinho\Downloads\Nova pasta\Leitor de Binários\shapes_deltaQQ")

# shapefile das sub-bacias
shp_sub = Path(r"E:\IGUAÇU_OTTO\Shp\Subbacias.shp")

# shapefile das sedes municipais (contém Curitiba e União da Vitória)
shp_cidades = Path(
    r"G:\Meu Drive\2_MESTRADO\1_Dissertação\Figuras\20250516_SHAPES_FIGURA\GEOFT_CIDADE_2016.shp"
)

# horizontes
horiz_codes  = ["2015_2040", "2041_2070", "2071_2100"]
horiz_labels = ["2015-2040", "2041-2070", "2071-2100"]

# pasta de saída
out = base_dir.parent / "figuras_3paineis_FINAL"
out.mkdir(exist_ok=True)

# ==============================
# 1. LISTAR MODELOS
# ==============================
modelos = sorted({
    "_".join(f.stem.split("_")[2:-2])
    for f in base_dir.glob("minis_deltaQQ_*.shp")
})

print(f"\nModelos detectados ({len(modelos)}):")
for m in modelos:
    print(" →", m)

# ==============================
# 2. DEFINIR ESCALA GLOBAL P/ TODOS MODELOS
# ==============================
valores = []
for modelo in modelos:
    for code in horiz_codes:
        df = gpd.read_file(base_dir / f"minis_deltaQQ_{modelo}_{code}.shp")
        valores.extend(df["delta_q_q"].tolist())

# escala fixa global (1º e 99º percentil para tirar extremos muito fora da curva)
vmin, vmax = np.percentile(valores, [1, 99])
print(f"\nESCALA GLOBAL FIXA APLICADA A TODOS MAPAS → {vmin:.1f}  a  {vmax:.1f}\n")

# ==============================
# 3. LER SUB-BACIAS E CIDADES
# ==============================
gdf_sub = gpd.read_file(shp_sub)
gdf_cid = gpd.read_file(shp_cidades)

print("\nCRS das cidades:", gdf_cid.crs)
print("Campos disponíveis nas cidades:", list(gdf_cid.columns))

# Campo com o nome do município (conforme a tabela)
CAMPO_NOME_CIDADE = "CID_NM"

# Filtro robusto por substring (case-insensitive), à prova de grafia
mask_cur = gdf_cid[CAMPO_NOME_CIDADE].str.contains("curit", case=False, na=False)
mask_un  = gdf_cid[CAMPO_NOME_CIDADE].str.contains("uni[aã]o da vit", case=False, na=False)

cidades_sel = gdf_cid[mask_cur | mask_un].copy()

print("\nCidades selecionadas para plotagem:")
print(cidades_sel[[CAMPO_NOME_CIDADE]].drop_duplicates())

# ==============================
# 4. MAPA 3×1 PARA CADA MODELO
# ==============================
for modelo in modelos:

    print(f"\nGerando figura → {modelo}")

    # ler shapefiles das minibacias para cada horizonte
    gdfs = [
        gpd.read_file(base_dir / f"minis_deltaQQ_{modelo}_{code}.shp")
        for code in horiz_codes
    ]

    # usar o CRS do primeiro shapefile como referência
    crs_minis = gdfs[0].crs

    # reprojetar sub-bacias e cidades para o mesmo CRS das minibacias
    gdf_sub_proj = gdf_sub.to_crs(crs_minis)
    cidades_proj = cidades_sel.to_crs(crs_minis) if not cidades_sel.empty else cidades_sel

    xmin, ymin, xmax, ymax = gdfs[0].total_bounds
    dx = (xmax - xmin) * 0.01  # deslocamento relativo (~1% da largura do mapa)
    dy = (ymax - ymin) * 0.01  # deslocamento relativo em y

    fig = plt.figure(figsize=(18, 6), dpi=300)
    gs  = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.05])

    axes = [fig.add_subplot(gs[i]) for i in range(3)]

    for ax, gdf, label in zip(axes, gdfs, horiz_labels):

        # mapa base das minibacias com ΔQ/Q
        gdf.plot(
            column="delta_q_q",
            cmap="RdYlBu_r",
            ax=ax,
            vmin=vmin,
            vmax=vmax,
            edgecolor="black",
            linewidth=0.08,
        )

        # contorno das sub-bacias (mais grosso para destacar)
        gdf_sub_proj.boundary.plot(
            ax=ax,
            edgecolor="grey",
            linewidth=1.2,
            zorder=3
        )

        # pontos das cidades (se o filtro encontrou alguma)
        if not cidades_proj.empty:
            cidades_proj.plot(
                ax=ax,
                marker="^",
                color="black",
                markersize=40,
                zorder=4,
                linewidth=0
            )

            # rótulos das cidades, com deslocamento relativo e halo
            for _, row in cidades_proj.iterrows():
                x = row.geometry.x
                y = row.geometry.y
                nome = row[CAMPO_NOME_CIDADE]

                txt = ax.text(
                    x + dx,
                    y + dy,
                    nome,
                    fontsize=9,
                    color="white",
                    ha="left",
                    va="bottom",
                    zorder=5,
                )
                txt.set_path_effects([
                    patheffects.Stroke(linewidth=1.5, foreground="black"),
                    patheffects.Normal()
                ])

        ax.set_title(f"Horizonte {label}", fontsize=11)
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal")
        ax.set_axis_off()

    # COLORBAR na coluna 4
    cax = fig.add_subplot(gs[3])
    sm  = plt.cm.ScalarMappable(
        cmap="RdYlBu_r",
        norm=plt.Normalize(vmin=vmin, vmax=vmax)
    )
    sm._A = []
    cbar = fig.colorbar(sm, cax=cax)
    cbar.set_label("ΔQ/Q (%)", fontsize=11)

    # título geral
    fig.suptitle(
        f"ΔQ/Q (%) – Modelo {modelo}",
        fontsize=14,
        weight="bold",
        y=0.97
    )

    # ajusta margens manualmente
    fig.subplots_adjust(left=0.02, right=0.97, top=0.90, bottom=0.03, wspace=0.02)

    # salvar figura
    fig.savefig(out / f"MAPA_3PAINEIS_FIXO_{modelo}.png")
    plt.close(fig)

print("\n✨ FINALIZADO – Todos os mapas gerados com sub-bacias e cidades destacadas.")
print(f"Pasta de saída: {out}")

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
from scipy.stats import kendalltau
import matplotlib.pyplot as plt
from matplotlib import gridspec

#=============================================================
# 1) CONFIGURAÇÃO DE DIRETÓRIOS
#=============================================================
base_txt  = Path(r"C:\Users\Matheus Marinho\Downloads\Nova pasta\Leitor de Binários")
base_shp  = base_txt / "shapes_deltaQQ"
output    = base_txt / "RESULTADOS_MK_FINAL"
output.mkdir(exist_ok=True)

#=============================================================
# 2) INTERVALOS TEMPORAIS (por ano)
#=============================================================
HORIZONTES = {
    "H1_2015_2040": (2015,2040),
    "H2_2041_2070": (2041,2070),
    "H3_2071_2100": (2071,2100),
    "TOTAL_2015_2100": (2015,2100),
}

P_LIMIT      = 0.05
DELTA_LIMIT  = 10.0
CONC_LIMIT   = 2/3

#=============================================================
# 3) CARREGA OBSERVADO PARA REFERÊNCIA (1980–2014)
#=============================================================
obs = pd.read_csv(base_txt/"Observado.txt", sep=r"\s+", header=None)
obs.columns=[f"mb_{i}" for i in range(1,obs.shape[1]+1)]
obs.index=pd.date_range("1980-01-01","2023-12-31",freq="D")

Qref = obs.loc["1980":"2023"].resample("A").mean().mean() # média anual histórica

#=============================================================
# 4) LOCALIZA MODELOS DISPONÍVEIS
#=============================================================
model_files = sorted([f for f in base_txt.glob("*.txt") if "Observado" not in f.name])
modelos=[f.stem for f in model_files]

print("\nMODELOS CARREGADOS:")
for m in modelos: print("→",m)

#=============================================================
# 5) FUNÇÃO MK UTILIZANDO SÉRIE ANUAL
#=============================================================
def mk_annual(series, y0, y1):
    ser = series[str(y0):str(y1)].resample("A").mean().dropna()
    if len(ser)<8: return np.nan
    t = np.arange(len(ser))
    tau,p = kendalltau(t,ser)
    return p

#=============================================================
# 6) PROCESSAMENTO GERAL
#=============================================================
resultado=[]

for modelo in modelos:

    print(f"\n⏳ PROCESSANDO {modelo} ...")
    df = pd.read_csv(base_txt/f"{modelo}.txt", sep=r"\s+", header=None)
    df.columns=Qref.index
    df.index=pd.date_range("2015-01-01","2100-12-31",freq="D")

    for mini in Qref.index:
        serie=df[mini]

        pvals  ={h:mk_annual(serie,ini,fim) for h,(ini,fim) in HORIZONTES.items()}
        deltas ={h:(serie[str(ini):str(fim)].resample("A").mean().mean()-Qref[mini])/
                  Qref[mini]*100 for h,(ini,fim) in HORIZONTES.items()}

        resultado.append({
            "modelo":modelo,"mini":mini,
            **{f"p_{h}":pvals[h] for h in HORIZONTES},
            **{f"dq_{h}":deltas[h] for h in HORIZONTES},
        })

resultado=pd.DataFrame(resultado)
resultado.to_csv(output/"MK_DeltaQ_Modelos.csv",index=False)
print("\n✔ Base anual completa salva.")

#=============================================================
# 7) CLASSIFICAÇÃO TERNÁRIA POR HORIZONTE
#=============================================================
classe={}

for h in HORIZONTES:

    df=resultado[["mini","modelo",f"p_{h}",f"dq_{h}"]]

    grup=df.groupby("mini").agg({
        f"p_{h}":lambda x:(x<P_LIMIT).mean(),
        f"dq_{h}":["mean",
                   lambda x:(x>+DELTA_LIMIT).mean(),
                   lambda x:(x<-DELTA_LIMIT).mean()]
    })

    grup.columns=["p_sig","dq_media","frac_pos","frac_neg"]
    grup["classe"]=0
    grup.loc[(grup["p_sig"]>=CONC_LIMIT)&(grup["frac_neg"]>=CONC_LIMIT)&(grup["dq_media"]<-DELTA_LIMIT),"classe"]=-2
    grup.loc[(grup["p_sig"]>=CONC_LIMIT)&(grup["frac_pos"]>=CONC_LIMIT)&(grup["dq_media"]>+DELTA_LIMIT),"classe"]=+2

    grup.to_csv(output/f"CLASS_{h}.csv")
    classe[h]=grup.reset_index()

print("\n✔ Critérios aplicados e salvos.")

#=============================================================
# 8) MAPAS FINAIS (3 HORIZONTES POR MODELO)
#=============================================================
shp=gpd.read_file(base_shp/f"minis_deltaQQ_{modelos[0]}_2015_2040.shp")

for modelo in modelos:
    print(f"\n🗺  MAPAS {modelo} ...")

    fig=plt.figure(figsize=(18,6),dpi=300)
    gs=gridspec.GridSpec(1,4,width_ratios=[1,1,1,0.07])

    for i,h in enumerate(["H1_2015_2040","H2_2041_2070","H3_2071_2100"]):

        df=classe[h]
        gdf=shp.merge(df,on="mini")

        ax=fig.add_subplot(gs[i])
        gdf.plot(column="classe",cmap="bwr",vmin=-2,vmax=2,
                 edgecolor="black",linewidth=0.08,ax=ax)
        ax.set_axis_off()
        ax.set_title(h.replace("_"," → ").replace("H1","2015–2040")
                     .replace("H2","2041–2070").replace("H3","2071–2100"))

    # barra de legenda
    cax=fig.add_subplot(gs[3])
    sm=plt.cm.ScalarMappable(cmap="bwr",norm=plt.Normalize(-2,2))
    sm._A=[]
    cbar=fig.colorbar(sm,cax=cax)
    cbar.set_ticks([-2,0,2])
    cbar.set_ticklabels(["Redução robusta","Inconclusivo","Aumento robusto"])

    plt.tight_layout()
    fig.savefig(output/f"MAPA_ROBUSTEZ_{modelo}.png")
    plt.close(fig)

print("\n🎉 FINALIZADO — RESULTADOS EM:\n",output)

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import gridspec, patheffects
from pathlib import Path

#==============================================================
# ⚙️ CAMINHOS
#==============================================================
base      = Path(r"C:\Users\Matheus Marinho\Downloads\Nova pasta\Leitor de Binários")

# shapefile das minibacias (onde está o ID_Mini)
shp_minis = Path(r"E:\IGUAÇU_OTTO\6_Calibração\minis_mgb.shp")

# shapefile das sub-bacias (contorno por sub-bacia)
shp_sub   = Path(r"E:\IGUAÇU_OTTO\Shp\Subbacias.shp")

# shapefile das sedes municipais (Curitiba e União da Vitória)
shp_cid   = Path(
    r"G:\Meu Drive\2_MESTRADO\1_Dissertação\Figuras\20250516_SHAPES_FIGURA\GEOFT_CIDADE_2016.shp"
)

out       = base / "RESULTADOS_Q95_Q5"
out_shp   = out  / "SHP"
out_maps  = out  / "MAPAS"
out.mkdir(exist_ok=True)
out_shp.mkdir(exist_ok=True)
out_maps.mkdir(exist_ok=True)

#==============================================================
# 📅 HORIZONTES (DIÁRIO) – MESMO PADRÃO DE NOMES DOS MAPAS DE MÉDIA
#==============================================================
H = {
    "2015_2040": ("2015-01-01", "2040-12-31"),
    "2041_2070": ("2041-01-01", "2070-12-31"),
    "2071_2100": ("2071-01-01", "2100-12-31"),
}
H_LABEL = {
    "2015_2040": "2015-2040",
    "2041_2070": "2041-2070",
    "2071_2100": "2071-2100",
}

#==============================================================
# Funções estatísticas
#==============================================================
def q95(s):
    return np.nanpercentile(s, 95)

def q5(s):
    return np.nanpercentile(s, 5)

#==============================================================
# 🔵 Calcular Qref do Observado (1980–2023)
#==============================================================
obs = pd.read_csv(base / "Observado.txt", sep=r"\s+", header=None)
obs.columns = [f"mb_{i}" for i in range(1, obs.shape[1] + 1)]
obs.index   = pd.date_range("1980-01-01", "2023-12-31", freq="D")

Qref95 = obs.loc["1980":"2023"].apply(q95)
Qref5  = obs.loc["1980":"2023"].apply(q5)

#==============================================================
# Listar modelos (todos .txt exceto Observado)
#==============================================================
modelos = [f.stem for f in base.glob("*.txt") if "Observado" not in f.name]

print("\nModelos encontrados:")
for m in modelos:
    print("  →", m)

#==============================================================
# 🔷 Ler shapes (minibacias, sub-bacias, cidades)
#==============================================================
shape_minis = gpd.read_file(shp_minis)
shape_sub   = gpd.read_file(shp_sub)
shape_cid   = gpd.read_file(shp_cid)

print("\nCRS das shapes:")
print("  Minibacias:", shape_minis.crs)
print("  Sub-bacias:", shape_sub.crs)
print("  Cidades   :", shape_cid.crs)

# Campo com o nome do município no shapefile de cidades
CAMPO_NOME_CIDADE = "CID_NM"

# Filtro robusto por substring (case-insensitive) para Curitiba e União da Vitória
mask_cur = shape_cid[CAMPO_NOME_CIDADE].str.contains("curit", case=False, na=False)
mask_un  = shape_cid[CAMPO_NOME_CIDADE].str.contains("uni[aã]o da vit", case=False, na=False)

cidades_sel = shape_cid[mask_cur | mask_un].copy()
print("\nCidades selecionadas:")
print(cidades_sel[[CAMPO_NOME_CIDADE]].drop_duplicates())

# Reprojetar sub-bacias e cidades para o CRS das minibacias
crs_minis     = shape_minis.crs
sub_proj      = shape_sub.to_crs(crs_minis)
cidades_proj  = cidades_sel.to_crs(crs_minis) if not cidades_sel.empty else cidades_sel

xmin, ymin, xmax, ymax = shape_minis.total_bounds
dx = (xmax - xmin) * 0.01  # deslocamento relativo p/ rótulos
dy = (ymax - ymin) * 0.01

#==============================================================
# 🔥 PRÉ-PASSO → Encontrar valores globais (para cor fixa)
#==============================================================
all_95_vals = []
all_5_vals  = []

for modelo in modelos:
    df = pd.read_csv(base / f"{modelo}.txt", sep=r"\s+", header=None)
    df.columns = Qref95.index
    df.index   = pd.date_range("2015-01-01", "2100-12-31", freq="D")

    for _, (ini, fim) in H.items():
        bloco = df.loc[ini:fim].copy()

        all_95_vals.extend(((bloco.apply(q95) - Qref95) / Qref95 * 100).values)
        all_5_vals.extend(((bloco.apply(q5) - Qref5) / Qref5 * 100).values)

# Escala global (pode usar min/max ou percentil; aqui uso percentil para tirar extremos)
vmin95, vmax95 = np.nanpercentile(all_95_vals, [1, 99])
vmin5 , vmax5  = np.nanpercentile(all_5_vals,  [1, 99])

print("\n📌 Escala global definida para todos os mapas:")
print(f"ΔQ95 → min={vmin95:.2f}   max={vmax95:.2f}")
print(f"ΔQ5  → min={vmin5 :.2f}   max={vmax5 :.2f}")

#==============================================================
# PROCESSAMENTO FINAL – SHPs + MAPAS
#==============================================================
for modelo in modelos:

    print(f"\n===========================\nPROCESSANDO {modelo}\n===========================")

    df = pd.read_csv(base / f"{modelo}.txt", sep=r"\s+", header=None)
    df.columns = Qref95.index
    df.index   = pd.date_range("2015-01-01", "2100-12-31", freq="D")

    deltaQ95 = {}
    deltaQ5  = {}

    #========== Q95 / Q5 por modelo ============================
    for Hcode, (ini, fim) in H.items():
        bloco = df.loc[ini:fim]

        dQ95 = (bloco.apply(q95) - Qref95) / Qref95 * 100
        dQ5  = (bloco.apply(q5)  - Qref5 ) / Qref5  * 100

        deltaQ95[Hcode] = dQ95
        deltaQ5 [Hcode] = dQ5

        #=== gerar SHP automático (minibacias) ===
        s95 = dQ95.copy()
        s95.index = s95.index.str.replace("mb_", "").astype(int)

        s5  = dQ5.copy()
        s5.index  = s5.index.str.replace("mb_", "").astype(int)

        g95 = shape_minis.merge(s95.rename("dQ95"), left_on="ID_Mini", right_index=True)
        g95.to_file(out_shp / f"Q95_{modelo}_{Hcode}.shp")

        g5  = shape_minis.merge(s5.rename("dQ5"), left_on="ID_Mini", right_index=True)
        g5.to_file(out_shp / f"Q5_{modelo}_{Hcode}.shp")

    #========== MAPA Q95 (escala global fixa) ==================
    fig = plt.figure(figsize=(18, 6), dpi=300)
    gs  = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.07])

    for i, (Hcode, serie) in enumerate(deltaQ95.items()):
        s = serie.copy()
        s.index = s.index.str.replace("mb_", "").astype(int)
        g = shape_minis.merge(s.rename("dQ95"), left_on="ID_Mini", right_index=True)

        ax = fig.add_subplot(gs[i])
        g.plot(
            column="dQ95",
            cmap="RdYlBu_r",
            vmin=vmin95,
            vmax=vmax95,
            edgecolor="black",
            linewidth=0.08,
            ax=ax
        )

        # contorno das sub-bacias
        sub_proj.boundary.plot(
            ax=ax,
            edgecolor="grey",
            linewidth=1.2,
            zorder=3
        )

        # cidades + rótulos
        if not cidades_proj.empty:
            cidades_proj.plot(
                ax=ax,
                marker="^",
                color="black",
                markersize=40,
                zorder=4,
                linewidth=0
            )

            for _, row in cidades_proj.iterrows():
                x = row.geometry.x
                y = row.geometry.y
                nome = row[CAMPO_NOME_CIDADE]

                txt = ax.text(
                    x + dx,
                    y + dy,
                    nome,
                    fontsize=9,
                    color="white",
                    ha="left",
                    va="bottom",
                    zorder=5,
                )
                txt.set_path_effects([
                    patheffects.Stroke(linewidth=1.5, foreground="black"),
                    patheffects.Normal()
                ])

        ax.set_title(f"Horizonte {H_LABEL[Hcode]}", fontsize=11)
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal")
        ax.set_axis_off()

    sm  = plt.cm.ScalarMappable(cmap="RdYlBu_r",
                                norm=plt.Normalize(vmin95, vmax95))
    sm._A = []
    cax = fig.add_subplot(gs[3])
    cbar = fig.colorbar(sm, cax=cax)
    cbar.set_label("ΔQ95 (%)")

    fig.suptitle(f"ΔQ95 (%) – Modelo {modelo}", fontsize=14, weight="bold", y=0.97)
    fig.subplots_adjust(left=0.02, right=0.97, top=0.90,
                        bottom=0.03, wspace=0.02)

    fig.savefig(out_maps / f"MAPA_Q95_{modelo}.png")
    plt.close(fig)
    print(f"📍 MAPA_Q95_{modelo}.png ✔")

    #========== MAPA Q5 (escala global fixa) ==================
    fig = plt.figure(figsize=(18, 6), dpi=300)
    gs  = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.07])

    for i, (Hcode, serie) in enumerate(deltaQ5.items()):
        s = serie.copy()
        s.index = s.index.str.replace("mb_", "").astype(int)
        g = shape_minis.merge(s.rename("dQ5"), left_on="ID_Mini", right_index=True)

        ax = fig.add_subplot(gs[i])
        g.plot(
            column="dQ5",
            cmap="RdYlBu_r",
            vmin=vmin5,
            vmax=vmax5,
            edgecolor="black",
            linewidth=0.08,
            ax=ax
        )

        # contorno das sub-bacias
        sub_proj.boundary.plot(
            ax=ax,
            edgecolor="grey",
            linewidth=1.2,
            zorder=3
        )

        # cidades + rótulos
        if not cidades_proj.empty:
            cidades_proj.plot(
                ax=ax,
                marker="^",
                color="black",
                markersize=40,
                zorder=4,
                linewidth=0
            )

            for _, row in cidades_proj.iterrows():
                x = row.geometry.x
                y = row.geometry.y
                nome = row[CAMPO_NOME_CIDADE]

                txt = ax.text(
                    x + dx,
                    y + dy,
                    nome,
                    fontsize=9,
                    color="white",
                    ha="left",
                    va="bottom",
                    zorder=5,
                )
                txt.set_path_effects([
                    patheffects.Stroke(linewidth=1.5, foreground="black"),
                    patheffects.Normal()
                ])

        ax.set_title(f"Horizonte {H_LABEL[Hcode]}", fontsize=11)
        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_aspect("equal")
        ax.set_axis_off()

    sm  = plt.cm.ScalarMappable(cmap="RdYlBu_r",
                                norm=plt.Normalize(vmin5, vmax5))
    sm._A = []
    cax = fig.add_subplot(gs[3])
    cbar = fig.colorbar(sm, cax=cax)
    cbar.set_label("ΔQ5 (%)")

    fig.suptitle(f"ΔQ5 (%) – Modelo {modelo}", fontsize=14, weight="bold", y=0.97)
    fig.subplots_adjust(left=0.02, right=0.97, top=0.90,
                        bottom=0.03, wspace=0.02)

    fig.savefig(out_maps / f"MAPA_Q5_{modelo}.png")
    plt.close(fig)
    print(f"📍 MAPA_Q5_{modelo}.png ✔")

print("\n🎉 FINALIZADO — SHPs e mapas de ΔQ95 e ΔQ5 gerados com o mesmo padrão dos mapas de ΔQ/Q.")

In [ ]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# ---------------------------------------------------
# CONFIGURAÇÕES
# ---------------------------------------------------
base_dir = Path(r"C:\Users\Matheus Marinho\Downloads\Nova pasta\Leitor de Binários")

obs_file = base_dir / "Observado.txt"
model_pattern = "*-pr-ssp245.txt"

ref_start = "1980-01-01"
ref_end   = "2023-12-31"

horizons = {
    "2015-2040": ("2015-01-01", "2040-12-31"),
    "2041-2070": ("2041-01-01", "2070-12-31"),
    "2071-2100": ("2071-01-01", "2100-12-31")
}

# 📁 saída — boxplots e tabela final
out = base_dir / "Q95_Q5_boxplots"
out.mkdir(exist_ok=True)

# ---------------------------------------------------
# Funções estatísticas
# ---------------------------------------------------
def q95(s): return np.nanpercentile(s,95)
def q5(s):  return np.nanpercentile(s,5)

# ---------------------------------------------------
# 1. Observado — Referência hidrológica
# ---------------------------------------------------
obs = pd.read_csv(obs_file, sep=r"\s+", header=None)
obs.columns = [f"mb_{i+1}" for i in range(obs.shape[1])]
obs.index   = pd.date_range("1980-01-01","2023-12-31",freq="D")

# Q95_ref e Q5_ref (base para o Δ%)
Q95_ref = obs.loc["1980":"2014"].apply(q95)
Q5_ref  = obs.loc["1980":"2014"].apply(q5)

# ---------------------------------------------------
# 2. Função principal – calcula ΔQ95 e ΔQ5 para um modelo
# ---------------------------------------------------
def processa_modelo(model_file):
    df = pd.read_csv(model_file, sep=r"\s+", header=None)
    df.columns = Q95_ref.index
    df.index   = pd.date_range("2015-01-01","2100-12-31",freq="D")
    model_name = model_file.stem

    registros = []

    for horiz_name,(h_start,h_end) in horizons.items():
        d = df.loc[h_start:h_end]

        Q95 = d.apply(q95)
        Q5  = d.apply(q5)

        dQ95 = (Q95 - Q95_ref) / Q95_ref * 100
        dQ5  = (Q5  - Q5_ref ) / Q5_ref  * 100

        # estrutura longa por minibacia
        tmp = pd.DataFrame({
            "modelo":model_name,
            "horizonte":horiz_name,
            "minibacia":dQ95.index,
            "delta_Q95":dQ95.values,
            "delta_Q5":dQ5.values
        })
        registros.append(tmp)

    return pd.concat(registros,ignore_index=True)

# ---------------------------------------------------
# 3. Loop geral — todos os modelos da pasta
# ---------------------------------------------------
resultados=[]

for mf in base_dir.glob(model_pattern):
    if "Observado" in mf.name: continue
    print(f"Processando modelo → {mf.name}")
    resultados.append(processa_modelo(mf))

df_final = pd.concat(resultados,ignore_index=True)
df_final.to_csv(out/"Q95_Q5_delta_por_modelo.csv",index=False)

# ---------------------------------------------------
# 4. BOXPLOTS — Q95 e Q5
# ---------------------------------------------------
modelos = sorted(df_final["modelo"].unique())

for horiz,(h_start,h_end) in horizons.items():

    df_h = df_final[df_final["horizonte"]==horiz]

    # BOX ΔQ95
    data95=[df_h[df_h.modelo==m]["delta_Q95"].values for m in modelos]
    plt.figure(figsize=(14,6))
    plt.boxplot(data95,labels=modelos,showfliers=False)
    plt.xticks(rotation=45,ha="right")
    plt.axhline(0,color="black",linestyle="--")
    plt.title(f"ΔQ95 (%) – distribuição por minibacia – {horiz}")
    plt.ylabel("ΔQ95 (%)")
    plt.tight_layout()
    plt.savefig(out/f"BOX_Q95_{horiz}.png",dpi=300)
    plt.close()

    # BOX ΔQ5
    data5=[df_h[df_h.modelo==m]["delta_Q5"].values for m in modelos]
    plt.figure(figsize=(14,6))
    plt.boxplot(data5,labels=modelos,showfliers=False)
    plt.xticks(rotation=45,ha="right")
    plt.axhline(0,color="black",linestyle="--")
    plt.title(f"ΔQ5 (%) – distribuição por minibacia – {horiz}")
    plt.ylabel("ΔQ5 (%)")
    plt.tight_layout()
    plt.savefig(out/f"BOX_Q5_{horiz}.png",dpi=300)
    plt.close()

print("\n📌 FINALIZADO — Boxplots e CSV salvos em:",out)

In [ ]:
"""
================================================================================
ENSEMBLE – Variação espacial de ΔQ/Q (%) por horizonte (SSP2-4.5 e SSP5-8.5)

FIGURA FINAL (2×1):
- Uma linha por cenário (SSP2-4.5 em cima, SSP5-8.5 embaixo)
- Em cada cenário: 4 boxplots lado a lado (um por horizonte) no MESMO eixo
- Eixo Y comum (comparável) e ajustado AUTOMATICAMENTE aos whiskers (quando showfliers=False)
- Linha de referência em ΔQ/Q = 0

TABELA COMPLETA:
- Estatísticas espaciais de ΔQ/Q (%) por cenário × horizonte:
  n_minibacias, min, p05, q1, mediana, media, q3, p95, max, iqr, amplitude,
  std, cv, skew, pct_pos, pct_neg, pct_zero
- Comparação espacial de níveis:
  Qpres_media_espacial, Qpres_mediana_espacial,
  Qfut_media_espacial, Qfut_mediana_espacial,
  Qfut_minus_Qpres_media_espacial, Qfut_minus_Qpres_mediana_espacial

Entradas:
- SSP2-4.5:
  E:\RESULTADOS\SSP2_45\Ensemble_Modelos\delta_q_q_ensemble_por_minibacia_ssp245.csv
- SSP5-8.5:
  E:\RESULTADOS\SSP5_85\Ensemble_Modelos\delta_q_q_ensemble_por_minibacia_ssp585.csv

Formato esperado do CSV:
['minibacia','horizonte','Q_pres','Q_fut_ens','delta_q_q_ensemble']
================================================================================
"""

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ==========================================================
# CONFIGURAÇÕES
# ==========================================================
ensemble_csv = {
    "SSP2-4.5": Path(r"E:\RESULTADOS_AB2\SSP2-45\Ensemble_Modelos\delta_q_q_ensemble_por_minibacia_ssp245.csv"),
    "SSP5-8.5": Path(r"E:\RESULTADOS_AB2\SSP5-85\Ensemble_Modelos\delta_q_q_ensemble_por_minibacia_ssp585.csv"),
}

out_dir = Path(r"E:\RESULTADOS_AB2\_FIGS_ENSEMBLE")
out_dir.mkdir(parents=True, exist_ok=True)

COL_ID  = "minibacia"
COL_H   = "horizonte"
COL_QP  = "Q_pres"
COL_QF  = "Q_fut_ens"
COL_DLT = "delta_q_q_ensemble"

HORIZ_ORDER = ["2015-2040", "2041-2070", "2071-2100", "2015-2100"]

# Figura final
FIG_W = 16
FIG_H = 10
SHOW_FLIERS = False   # se True, plota outliers como bolinhas
WHIS = 1.5            # whiskers padrão (IQR*1.5)
Y_PAD_FRAC = 0.08     # margem visual


# ==========================================================
# FUNÇÕES
# ==========================================================
def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.replace("\ufeff", "", regex=False)
        .str.replace("\u200b", "", regex=False)
        .str.strip()
    )
    return df

def ensure_cols(df: pd.DataFrame, required: list[str], scen: str):
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"[{scen}] Colunas ausentes: {missing}\nColunas: {list(df.columns)}")

def build_box_data(df: pd.DataFrame):
    df = df.copy()
    df[COL_H] = df[COL_H].astype(str).str.strip()

    horizons = [h for h in HORIZ_ORDER if h in set(df[COL_H])]
    if not horizons:
        horizons = list(df[COL_H].unique())

    data = []
    for h in horizons:
        v = pd.to_numeric(df.loc[df[COL_H] == h, COL_DLT], errors="coerce").dropna()
        data.append(v.values)

    return horizons, data

def whisker_limits(values: np.ndarray, whis: float = 1.5):
    v = pd.to_numeric(pd.Series(values), errors="coerce").dropna().values
    if v.size == 0:
        return np.nan, np.nan

    q1 = np.quantile(v, 0.25)
    q3 = np.quantile(v, 0.75)
    iqr = q3 - q1

    lf = q1 - whis * iqr
    uf = q3 + whis * iqr

    v_in = v[(v >= lf) & (v <= uf)]
    if v_in.size == 0:
        return float(np.min(v)), float(np.max(v))

    return float(np.min(v_in)), float(np.max(v_in))

def compute_global_ylim(data_by_scen: dict):
    """
    y-limits globais coerentes entre cenários.
    - Se SHOW_FLIERS=False: usa whiskers (ignora outliers não plotados)
    - Se SHOW_FLIERS=True: usa min/max total (porque outliers aparecem)
    """
    lows, highs = [], []

    for data_list in data_by_scen.values():
        for arr in data_list:
            if arr is None or len(arr) == 0:
                continue

            if SHOW_FLIERS:
                v = pd.to_numeric(pd.Series(arr), errors="coerce").dropna().values
                if v.size == 0:
                    continue
                lo, hi = float(np.min(v)), float(np.max(v))
            else:
                lo, hi = whisker_limits(arr, whis=WHIS)

            if not np.isnan(lo) and not np.isnan(hi):
                lows.append(lo)
                highs.append(hi)

    if not lows:
        return (-1.0, 1.0)

    y_min = min(min(lows), 0.0)
    y_max = max(max(highs), 0.0)

    span = (y_max - y_min) if y_max > y_min else 1.0
    pad = Y_PAD_FRAC * span
    return (y_min - pad, y_max + pad)

def spatial_stats_full(values: pd.Series) -> dict:
    """
    Estatísticas completas da variação espacial de ΔQ/Q (%).
    """
    v = pd.to_numeric(values, errors="coerce").dropna()
    if v.empty:
        return {
            "n_minibacias": 0,
            "min": np.nan, "p05": np.nan, "q1": np.nan, "mediana": np.nan, "media": np.nan,
            "q3": np.nan, "p95": np.nan, "max": np.nan, "iqr": np.nan, "amplitude": np.nan,
            "std": np.nan, "cv": np.nan, "skew": np.nan,
            "pct_pos": np.nan, "pct_neg": np.nan, "pct_zero": np.nan,
        }

    q1 = v.quantile(0.25)
    q3 = v.quantile(0.75)
    media = v.mean()
    std = v.std(ddof=1)
    cv = (std / abs(media)) if media != 0 else np.nan

    return {
        "n_minibacias": int(v.shape[0]),
        "min": float(v.min()),
        "p05": float(v.quantile(0.05)),
        "q1": float(q1),
        "mediana": float(v.median()),
        "media": float(media),
        "q3": float(q3),
        "p95": float(v.quantile(0.95)),
        "max": float(v.max()),
        "iqr": float(q3 - q1),
        "amplitude": float(v.max() - v.min()),
        "std": float(std),
        "cv": float(cv),
        "skew": float(v.skew()),
        "pct_pos": float((v > 0).mean() * 100.0),
        "pct_neg": float((v < 0).mean() * 100.0),
        "pct_zero": float((v == 0).mean() * 100.0),
    }

def q_spatial_summary(df_h: pd.DataFrame) -> dict:
    qp = pd.to_numeric(df_h[COL_QP], errors="coerce").dropna()
    qf = pd.to_numeric(df_h[COL_QF], errors="coerce").dropna()
    return {
        "Qpres_media_espacial": float(qp.mean()) if not qp.empty else np.nan,
        "Qpres_mediana_espacial": float(qp.median()) if not qp.empty else np.nan,
        "Qfut_media_espacial": float(qf.mean()) if not qf.empty else np.nan,
        "Qfut_mediana_espacial": float(qf.median()) if not qf.empty else np.nan,
    }


# ==========================================================
# LEITURA
# ==========================================================
dfs = {}
boxdata_by_scen = {}

for scen, path in ensemble_csv.items():
    if not path.exists():
        raise FileNotFoundError(f"[{scen}] CSV não encontrado: {path}")

    df = pd.read_csv(path, sep=None, engine="python")
    df = clean_columns(df)
    ensure_cols(df, [COL_ID, COL_H, COL_QP, COL_QF, COL_DLT], scen)

    df[COL_ID] = df[COL_ID].astype(str).str.strip()
    df[COL_H] = df[COL_H].astype(str).str.strip()

    dfs[scen] = df

    _, data = build_box_data(df)
    boxdata_by_scen[scen] = data

ylims = compute_global_ylim(boxdata_by_scen)
print("[INFO] y-limits globais:", ylims)


# ==========================================================
# FIGURA FINAL (2×1) – UM EIXO POR CENÁRIO, 4 BOXPLOTS NO MESMO EIXO
# ==========================================================
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(FIG_W, FIG_H), dpi=300, sharey=True)
axes = np.array(axes).reshape(-1)

for ax, scen in zip(axes, ["SSP2-4.5", "SSP5-8.5"]):
    labels, data = build_box_data(dfs[scen])

    ax.boxplot(data, labels=labels, showfliers=SHOW_FLIERS, whis=WHIS)
    ax.axhline(0, linestyle="--", linewidth=1, color="grey")
    ax.set_ylim(*ylims)

    ax.set_ylabel("ΔQ/Q (%)")
    ax.set_title(f"{scen} — Ensemble: variação espacial de ΔQ/Q (%) por horizonte")

axes[-1].set_xlabel("Horizonte")
plt.tight_layout()

fig_out = out_dir / "painel_final_ensemble_2x1_quatro_horizontes.png"
plt.savefig(fig_out, dpi=300, bbox_inches="tight")
plt.close(fig)

print("Figura salva:", fig_out)


# ==========================================================
# TABELA COMPLETA DE ESTATÍSTICAS ESPACIAIS
# ==========================================================
rows = []

for scen, df in dfs.items():
    # garante ordem de horizontes
    for h in HORIZ_ORDER:
        sub = df[df[COL_H] == h].copy()
        if sub.empty:
            continue

        st = spatial_stats_full(sub[COL_DLT])
        qs = q_spatial_summary(sub)

        rows.append({
            "cenario": scen,
            "horizonte": h,
            **st,
            **qs,
            "Qfut_minus_Qpres_media_espacial": (
                qs["Qfut_media_espacial"] - qs["Qpres_media_espacial"]
                if (not np.isnan(qs["Qfut_media_espacial"]) and not np.isnan(qs["Qpres_media_espacial"]))
                else np.nan
            ),
            "Qfut_minus_Qpres_mediana_espacial": (
                qs["Qfut_mediana_espacial"] - qs["Qpres_mediana_espacial"]
                if (not np.isnan(qs["Qfut_mediana_espacial"]) and not np.isnan(qs["Qpres_mediana_espacial"]))
                else np.nan
            ),
        })

stats_df = pd.DataFrame(rows)

# ordenação
stats_df["horiz_ord"] = pd.Categorical(stats_df["horizonte"], categories=HORIZ_ORDER, ordered=True)
stats_df = stats_df.sort_values(["cenario", "horiz_ord"]).drop(columns=["horiz_ord"])

out_stats = out_dir / "estatisticas_espaciais_ensemble_por_horizonte.csv"
stats_df.to_csv(out_stats, index=False, encoding="utf-8-sig")
print("Tabela salva:", out_stats)

In [ ]:
"""
================================================================================
Mapa da ANOMALIA ABSOLUTA entre Abordagem 2 e Abordagem 1 (m³/s), por minibacia
================================================================================

ANOMALIA_ABS (m³/s) = Q_AB2 - Q_AB1

ATUALIZAÇÕES (LEGIBILIDADE)
---------------------------
1) Escala DISCRETA por CLASSES FIXAS (bins) — sem rampa contínua
2) Mesma legenda/escala para todos os painéis
3) Gera também uma figura SEPARADA apenas para o período TOTAL (2015–2100)
4) SUBTÍTULOS sem sobreposição: desenhados fora do mapa com ax.text (transAxes)

================================================================================
"""

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import patheffects
from matplotlib.colors import ListedColormap, BoundaryNorm
from pathlib import Path
import numpy as np

# ==============================================================================
# 0) CONFIGURAÇÕES – AJUSTE AQUI
# ==============================================================================

# --- Caminhos dos CSVs por abordagem ---
CSV_AB1 = Path(r"E:\RESULTADOS\SSP5_85\Ensemble_Modelos\delta_q_q_ensemble_por_minibacia_ssp585.csv")
CSV_AB2 = Path(r"E:\RESULTADOS_AB2\SSP5-85\Ensemble_Modelos\delta_q_q_ensemble_por_minibacia_ssp585.csv")

# --- Nome da coluna com a vazão típica futura do ensemble (m³/s) ---
COL_QFUT = "Q_fut_ens"

# --- Horizonte(s) a mapear ---
HORIZ_LABELS = ["2015-2040", "2041-2070", "2071-2100", "2015-2100"]
HORIZ_CODES  = ["2015_2040", "2041_2070", "2071_2100", "2015_2100"]

# --- Layout do mapa final ---
# "4" = 2x2 (inclui Total), "3" = 3x1 (sem Total)
LAYOUT = "4"

# --- Shapefile das minibacias ---
SHP_MINIS = Path(r"E:\IGUAÇU_OTTO\6_Calibração\minis_mgb.shp")
ID_FIELD  = "ID_Mini"

# --- Camadas auxiliares (opcionais) ---
SHP_SUB = Path(r"E:\IGUAÇU_OTTO\Shp\Subbacias.shp")
SHP_CIDADES = Path(r"G:\Meu Drive\2_MESTRADO\1_Dissertação\Figuras\20250516_SHAPES_FIGURA\GEOFT_CIDADE_2016.shp")
CAMPO_NOME_CIDADE = "CID_NM"

# --- Saídas ---
OUT_DIR = Path(r"E:\IGUAÇU_OTTO\9_Figuras_e_Tabelas_Ensemble_AB2\Anomalia_AB2_minus_AB1")
OUT_DIR.mkdir(parents=True, exist_ok=True)

SHAPES_DIR = OUT_DIR / "shapes_anomalia"
SHAPES_DIR.mkdir(exist_ok=True)

FIG_DIR = OUT_DIR / "figuras"
FIG_DIR.mkdir(exist_ok=True)

# ------------------------------------------------------------------------------
# ESCALA DISCRETA (BINS FIXOS)
# ------------------------------------------------------------------------------
BINS_FIXED = [-40, -30, -20, -12, -6, -2, 2, 6, 12, 20, 30, 40]
CMAP_NAME = "RdYlBu"

# ------------------------------------------------------------------------------
# AJUSTES DE LAYOUT PARA SUBTÍTULOS (SEM SOBREPOR)
# ------------------------------------------------------------------------------
SUBTITLE_Y = 1.04         # posição do subtítulo acima do eixo (transAxes)
SUBTITLE_FONTSIZE = 11
HSPACE_2X2 = 0.12         # espaço vertical entre linhas no GridSpec 2x2


# ==============================================================================
# 1) FUNÇÕES AUXILIARES
# ==============================================================================

def read_and_prepare(csv_path: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    req = {"minibacia", "horizonte", COL_QFUT}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(
            f"CSV {csv_path.name} não contém colunas obrigatórias: {missing}. "
            f"Colunas: {list(df.columns)}"
        )

    # mini: "mb_1" -> 1
    df["mini"] = df["minibacia"].astype(str).str.replace("mb_", "", regex=False).astype(int)
    df["horizonte"] = df["horizonte"].astype(str)

    df = df[["mini", "horizonte", COL_QFUT]].copy()
    df = df.rename(columns={COL_QFUT: "Q_fut"})
    return df


def compute_anomaly(df_ab1: pd.DataFrame, df_ab2: pd.DataFrame) -> pd.DataFrame:
    """
    Retorna tabela com:
      mini, horizonte, Q_ab1, Q_ab2, anomalia_abs (m³/s)
    """
    m = df_ab1.merge(df_ab2, on=["mini", "horizonte"], how="inner", suffixes=("_ab1", "_ab2"))
    m = m.rename(columns={"Q_fut_ab1": "Q_ab1", "Q_fut_ab2": "Q_ab2"})
    m["anomalia_abs"] = m["Q_ab2"] - m["Q_ab1"]
    return m


def build_discrete_style(bins: list[float], cmap_name: str):
    """
    Cria cmap discreto + norm por classes (BoundaryNorm).
    """
    n_classes = len(bins) - 1
    base = plt.get_cmap(cmap_name)
    colors = base(np.linspace(0, 1, n_classes))
    cmap = ListedColormap(colors)
    norm = BoundaryNorm(bins, ncolors=n_classes, clip=True)
    return cmap, norm


def plot_colorbar_discrete(fig, cax, cmap, norm, bins):
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm._A = []

    # ticks nos limites das classes
    cbar = fig.colorbar(sm, cax=cax, ticks=bins, boundaries=bins, spacing="proportional")
    cbar.set_label("Anomalia (m³/s)", fontsize=11)
    cbar.ax.tick_params(labelsize=9)


def plot_panel(ax, gdf, title, gdf_sub, cidades, bounds, cmap, norm):
    """
    Painel do mapa + subtítulo fora do mapa (sem sobrepor).
    """
    xmin, ymin, xmax, ymax = bounds
    dx = (xmax - xmin) * 0.01
    dy = (ymax - ymin) * 0.01

    gdf.plot(
        column="anomalia_abs",
        cmap=cmap,
        norm=norm,
        ax=ax,
        edgecolor="black",
        linewidth=0.08,
    )

    if gdf_sub is not None:
        gdf_sub.boundary.plot(ax=ax, edgecolor="grey", linewidth=1.2, zorder=3)

    if cidades is not None and not cidades.empty:
        cidades.plot(ax=ax, marker="^", color="black", markersize=40, zorder=4, linewidth=0)

        for _, row in cidades.iterrows():
            x = row.geometry.x
            y = row.geometry.y
            nome = row[CAMPO_NOME_CIDADE]
            txt = ax.text(
                x + dx, y + dy, nome,
                fontsize=9, color="white",
                ha="left", va="bottom", zorder=5
            )
            txt.set_path_effects([
                patheffects.Stroke(linewidth=1.5, foreground="black"),
                patheffects.Normal()
            ])

    # --- Subtítulo fora do mapa (posição relativa ao eixo) ---
    ax.text(
        0.5, SUBTITLE_Y, title,
        transform=ax.transAxes,
        ha="center", va="bottom",
        fontsize=SUBTITLE_FONTSIZE,
        bbox=dict(facecolor="white", edgecolor="none", pad=2.5),
        clip_on=False,
        zorder=10
    )

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_aspect("equal")
    ax.set_axis_off()


def save_total_only_map(gdf_total, bounds, gdf_sub, cidades_sel, cmap, norm, bins, out_path: Path, suptitle: str):
    """
    Figura individual apenas para o horizonte total (2015-2100).
    """
    fig = plt.figure(figsize=(9, 6), dpi=300)
    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 0.06], wspace=0.03)

    ax = fig.add_subplot(gs[0, 0])
    cax = fig.add_subplot(gs[0, 1])

    plot_panel(
        ax=ax,
        gdf=gdf_total,
        title="Horizonte 2015–2100",
        gdf_sub=gdf_sub,
        cidades=cidades_sel,
        bounds=bounds,
        cmap=cmap,
        norm=norm
    )

    plot_colorbar_discrete(fig, cax, cmap, norm, bins)

    fig.suptitle(suptitle, fontsize=13, weight="bold", y=0.98)
    fig.tight_layout(rect=[0.03, 0.03, 0.97, 0.95])
    fig.savefig(out_path, dpi=300)
    plt.close(fig)


# ==============================================================================
# 2) PIPELINE PRINCIPAL
# ==============================================================================

print("\n=== Lendo CSVs e calculando anomalia AB2 − AB1 ===\n")

df1 = read_and_prepare(CSV_AB1)
df2 = read_and_prepare(CSV_AB2)

df_anom = compute_anomaly(df1, df2)

# Filtrar horizontes desejados
df_anom = df_anom[df_anom["horizonte"].isin(HORIZ_LABELS)].copy()

print("Horizontes disponíveis:", sorted(df_anom["horizonte"].unique()))
print("Minibacias:", df_anom["mini"].nunique())

# Ler shapefile minibacias
minis = gpd.read_file(SHP_MINIS)
minis[ID_FIELD] = minis[ID_FIELD].astype(int)

# Camadas auxiliares
gdf_sub = None
cidades_sel = None

# Preparar CRS base (minibacias)
crs_minis = minis.crs

if SHP_SUB.exists():
    gdf_sub = gpd.read_file(SHP_SUB).to_crs(crs_minis)

if SHP_CIDADES.exists():
    gdf_cid = gpd.read_file(SHP_CIDADES).to_crs(crs_minis)
    mask_cur = gdf_cid[CAMPO_NOME_CIDADE].str.contains(r"^curitiba$", case=False, na=False, regex=True)
    mask_un  = gdf_cid[CAMPO_NOME_CIDADE].str.contains("uni[aã]o da vit", case=False, na=False, regex=True)
    cidades_sel = gdf_cid[mask_cur | mask_un].copy()

# Gerar shapefiles por horizonte + coletar valores
print("\n=== Gerando shapefiles temáticos por horizonte ===\n")

gdfs_h = []
values_all = []

for h_label, h_code in zip(HORIZ_LABELS, HORIZ_CODES):
    df_h = df_anom[df_anom["horizonte"] == h_label][["mini", "anomalia_abs"]].copy()

    gdf_h = minis.merge(df_h, left_on=ID_FIELD, right_on="mini", how="left")
    shp_out = SHAPES_DIR / f"minis_anomalia_ab2_minus_ab1_{h_code}.shp"
    gdf_h.to_file(shp_out)

    print(f"Shapefile salvo: {shp_out.name}")

    gdfs_h.append((h_label, gdf_h))
    values_all.extend(gdf_h["anomalia_abs"].dropna().tolist())

if not values_all:
    raise RuntimeError("Não encontrei valores de anomalia para plotagem.")

# Estilo discreto (escala fixa por bins)
bins = BINS_FIXED
cmap, norm = build_discrete_style(bins, CMAP_NAME)

# Bounds para manter mesmo enquadramento em todos os painéis
bounds = minis.total_bounds

# ==============================================================================
# 3) FIGURA FINAL (3 ou 4 painéis) — CLASSES FIXAS + SUBTÍTULO SEM SOBREPOR
# ==============================================================================

if LAYOUT == "3":
    # 3×1 (sem Total)
    fig = plt.figure(figsize=(18, 6), dpi=300)
    gs  = gridspec.GridSpec(1, 4, width_ratios=[1, 1, 1, 0.06], wspace=0.05)

    axes = [fig.add_subplot(gs[i]) for i in range(3)]
    cax = fig.add_subplot(gs[3])

    for ax, (h_label, gdf_h) in zip(axes, gdfs_h[:3]):
        plot_panel(ax, gdf_h, f"Horizonte {h_label}", gdf_sub, cidades_sel, bounds, cmap, norm)

    plot_colorbar_discrete(fig, cax, cmap, norm, bins)

    fig.suptitle("Anomalia absoluta de vazão (m³/s) – SSP5-8.5", fontsize=14, weight="bold", y=0.98)
    fig.subplots_adjust(left=0.05, right=0.95, top=0.88, bottom=0.03)

    fig_path = FIG_DIR / "MAPA_ANOMALIA_ABS_AB2_minus_AB1_3paineis_classes.png"
    fig.savefig(fig_path, dpi=300)
    plt.close(fig)

else:
    # 2×2 (4 painéis, inclui Total)
    fig = plt.figure(figsize=(16, 10), dpi=300)

    # hspace maior para "abrir" espaço entre as linhas e acomodar subtítulos fora do mapa
    gs  = gridspec.GridSpec(
        2, 3,
        width_ratios=[1, 1, 0.08],
        wspace=0.02,
        hspace=HSPACE_2X2
    )

    ax_11 = fig.add_subplot(gs[0, 0])
    ax_12 = fig.add_subplot(gs[0, 1])
    ax_21 = fig.add_subplot(gs[1, 0])
    ax_22 = fig.add_subplot(gs[1, 1])
    cax   = fig.add_subplot(gs[:, 2])

    axes = [ax_11, ax_12, ax_21, ax_22]

    for ax, (h_label, gdf_h) in zip(axes, gdfs_h):
        plot_panel(ax, gdf_h, f"Horizonte {h_label}", gdf_sub, cidades_sel, bounds, cmap, norm)

    plot_colorbar_discrete(fig, cax, cmap, norm, bins)

    fig.suptitle("Anomalia absoluta de vazão (m³/s) – SSP5-8.5", fontsize=14, weight="bold", y=0.985)

    # reservar topo maior para o título geral + subtítulos
    fig.tight_layout(rect=[0.02, 0.02, 0.98, 0.90])

    fig_path = FIG_DIR / "MAPA_ANOMALIA_ABS_AB2_minus_AB1_4paineis_classes.png"
    fig.savefig(fig_path, dpi=300)
    plt.close(fig)

# ==============================================================================
# 4) FIGURA SEPARADA — APENAS PERÍODO TOTAL (2015–2100)
# ==============================================================================

gdf_total = None
for h_label, gdf_h in gdfs_h:
    if h_label == "2015-2100":
        gdf_total = gdf_h
        break

if gdf_total is not None:
    fig_total_path = FIG_DIR / "MAPA_ANOMALIA_ABS_AB2_minus_AB1_TOTAL_2015_2100_classes.png"
    save_total_only_map(
        gdf_total=gdf_total,
        bounds=bounds,
        gdf_sub=gdf_sub,
        cidades_sel=cidades_sel,
        cmap=cmap,
        norm=norm,
        bins=bins,
        out_path=fig_total_path,
        suptitle="Anomalia absoluta de vazão (m³/s) – SSP5-8.5"
    )
else:
    fig_total_path = None
    print("\n⚠️ Horizonte 2015-2100 não encontrado nos dados filtrados; não foi possível gerar a figura total.")

print("\n✅ Figuras geradas com sucesso!")
print("→ Figura principal:", fig_path)
if fig_total_path is not None:
    print("→ Figura total (2015–2100):", fig_total_path)
print("Shapefiles:", SHAPES_DIR)